In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_columns=None
pd.options.display.max_colwidth=300
import matplotlib.pyplot as plt
import seaborn as sns

### Loading the data
Dataset 'Encuesta_insercion_laboral_titulados_universitarios_2019' en formato csv.   

Enlace a fuente (INE): https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176991&menu=resultados&idp=1254735573113#!tabs-1254736195339

In [2]:
df_grad=pd.read_csv('/home/bea/Escritorio/KSchool_TFM/Principales/Encuesta_insercion_laboral_titulados_universitarios_2019/EILU_GRADUADOS/CSV/EILU_GRAD_2019.csv',
                    sep='\t')

/home/bea/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (119) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_grad.shape

(31651, 299)

In [4]:
for col in df_grad.columns:
    print(col,df_grad[col].dtype)

IDENT int64
SEXO int64
EDAD int64
NACIO int64
NACIO1 object
NACIO2 object
PAIS_NACI int64
TITU int64
RAMA int64
AMBITO int64
T_UNIV int64
DISCA int64
PROXY int64
LUG_RES_PAIS int64
PAIS_NAC_PADRE int64
ESTUDIOS_PADRE int64
PAIS_NAC_MADRE int64
ESTUDIOS_MADRE int64
EST_B1 int64
EST_B2_1 object
EST_B2_2 object
EST_B2_3 object
EST_B2_4 object
EST_B2_5 object
EST_M1 int64
EST_M2 object
EST_M3 object
EST_M4 object
EST_MES int64
EST_B8 int64
SAT1 int64
SAT2 object
EST_B11_1 int64
EST_B11_2 int64
EST_B11_3 int64
EST_B11_4 int64
EST_B11_5 int64
EST_B11_6 int64
EST_B11_7 int64
EST_B12 object
EST_B13_AMB1 object
EST_B14_1 object
EST_B15_1 object
EST_B13_AMB2 object
EST_B14_2 object
EST_B15_2 object
EST_B13_AMB3 object
EST_B14_3 object
EST_B15_3 object
EST_B16 object
EST_B17_M1 object
EST_B18_1 object
EST_B19_1 object
EST_B17_M2 object
EST_B18_2 object
EST_B19_2 object
EST_B17_M3 object
EST_B18_3 object
EST_B19_3 object
EST_B21 object
EST_B22 object
EST_B23 object
EST_B24 object
EST_B25_FA1 objec

### Análisis descriptivo: funciones 
Funciones definidas para facilitar la inspección de variables categóricas: distribución de frecuencias relativas, distribuciones condicionadas, etc. 

In [5]:
def proporciones(col,dropna=True):
    """ Frecuencias relativas de los valores que toma una variable categórica en orden ascendente.
    Si dropna=False, incluye los valores NaN."""
    if dropna==True: 
        out=df_grad[col].value_counts(normalize=True,ascending=True)
    else: 
        out=df_grad[col].value_counts(normalize=True,ascending=True,dropna=False)
    return out

In [6]:
def dist_condicionada(col,cond,rev=False):
    """ Distribución de la columna col condicionada por la columna cond. Si rev=True, distribución de 
    cond condicionada por col."""
    if rev==False: 
        out=df_grad[[cond,col]].groupby([cond,col]).agg({col:'count'}
                ).div(df_grad[[cond,col]].groupby(cond).agg({col:'count'}),level=cond)
    else: 
        out=df_grad[[col,cond]].groupby([col,cond]).agg({cond:'count'}
                ).div(df_grad[[col,cond]].groupby(col).agg({cond:'count'}),level=col)
    return out

### Inspección y ajustes
Inspección del dataset, imputación de valores perdidos debido a los itinerarios de la encuesta y creación de variables nuevas para análisis descriptivo y predictivo. 

In [7]:
# Lista a la que se agregarán las variables incluidas en el modelo predictivo
Variables=list()

#### 'IDENT'
**Índice** único del entrevistado.

In [8]:
df_grad['IDENT'].unique()

array([    1,     2,     3, ..., 31649, 31650, 31651])

#### 'SEXO' (redefinida como 'fem')
**Sexo**: '1'=Hombre, '2'='Mujer'.  


Reasignación de valores **'fem'**: 1= Mujer, 0=Hombre. 

In [9]:
df_grad['SEXO'].unique() # No missing values

array([1, 2])

In [10]:
df_grad['fem']=0
df_grad['fem'].mask(df_grad['SEXO']==2,1,inplace=True)
df_grad[['SEXO','fem']].groupby('fem').agg(['unique']) # Comprobación

,SEXO
,unique
fem,
0,[1]
1,[2]


In [11]:
Variables.append('fem')

In [12]:
proporciones('fem')

0    0.429528
1    0.570472
Name: fem, dtype: float64

#### 'EDAD' (redesignada como 'edad')
**Edad** del entrevistado: 1=Menores de 30 años, 2=De 30 a 34 años, 3=De 35 y más años. 

In [13]:
df_grad['EDAD'].unique() # No missing values

array([1, 2, 3])

In [14]:
df_grad.rename(columns={'EDAD':'edad'},inplace=True)

In [15]:
Variables.append('edad')

In [16]:
proporciones('edad')

3    0.224922
2    0.278822
1    0.496256
Name: edad, dtype: float64

#### 'NACIO'
**Nacionalidad** del entrevistado: 1=Española, 2=Española y otra, 3=Otra nacionalidad

In [17]:
df_grad['NACIO'].unique() # No missing values

array([1, 2, 3])

In [18]:
proporciones('NACIO')

2    0.010837
3    0.011027
1    0.978137
Name: NACIO, dtype: float64

#### Nueva variable 'esp'
**Nacionalidad española** u otra: 0=Nacionalidad no española, 1=Nacionalidad española. 

In [19]:
df_grad['esp']=1
df_grad['esp'].mask(df_grad['NACIO']==3,0,inplace=True)
df_grad[['esp','NACIO']].groupby(['NACIO']).agg(['unique']) # Comprobación

,esp
,unique
NACIO,
1,[1]
2,[1]
3,[0]


In [20]:
Variables.append('esp')

In [21]:
proporciones('esp')

0    0.011027
1    0.988973
Name: esp, dtype: float64

In [22]:
# Distribución condicionada de sexos condicionada por nacionalidad española u otra: 
dist_condicionada('fem','esp')
# La mayor presencia femenina entre los graduados se acentúa en el caso quienes no tienen nacionalidad española

fem
esp fem          
0   0    0.343840
    1    0.656160
1   0    0.430484
    1    0.569516

#### 'NACIO1'
**País de primera nacionalidad**: 1=Un país de la UE, 2=Un país fuera de la UE, 9=NS/NC.

In [23]:
df_grad['NACIO1'].unique() # Missing values as ' ' or '9'

array([' ', '2', '9', '1'], dtype=object)

In [24]:
df_grad[['NACIO','NACIO1']].groupby('NACIO').agg(['unique'])
# Respuesta en blanco para quienes declararon solo nacionalidad española en 'NACIO'

,NACIO1
,unique
NACIO,
1,[ ]
2,"[2, 1, 9]"
3,"[9, 2, 1]"


In [25]:
# Se imputa nacionalidad de un país de la UE (1) a quien declaró solo nacionalidad española en variable 'NACIO'
df_grad['NACIO1'].mask(df_grad['NACIO']==1,1,inplace=True)
# Se imputa None a respuesta en blanco y a NS/NC (9)
df_grad['NACIO1'].replace({' ':None,'9':None},inplace=True)
df_grad['NACIO1']=df_grad['NACIO1'].astype('float')
df_grad[['NACIO','NACIO1']].groupby('NACIO').agg(['unique'])

,NACIO1
,unique
NACIO,
1,[1.0]
2,"[2.0, 1.0, nan]"
3,"[nan, 2.0, 1.0]"


In [26]:
proporciones('NACIO1',dropna=False)

NaN    0.001738
2.0    0.012511
1.0    0.985751
Name: NACIO1, dtype: float64

#### 'NACIO2'
**País de segunda nacionalidad**: 1=Un país de la UE, 2=Un país fuera de la UE, 9=NS/NC.  

Nota: Se añade valor 0=Sin segunda nacionalidad.

In [27]:
df_grad['NACIO2'].unique() # Missing values as ' '

array([' ', '1', '2'], dtype=object)

In [28]:
df_grad[['NACIO','NACIO2']].groupby('NACIO').agg(['unique'])
# Sin respuesta para quienes declaran 'nacionalidad española' y 'nacionalidad española y otra' (1 y 2 en 'NACIO').

,NACIO2
,unique
NACIO,
1,[ ]
2,[ ]
3,"[ , 1, 2]"


In [29]:
# Se imputa 'NACIO2'=1 (UE)  a quienes declararon 'nacionalidad española y otra' (2 en 'NACIO') y la primera ...
# ... no es nacionalidad EU ('NACIO1' distinta de 1), ya que la española es su segunda nacionalidad:
df_grad['NACIO2'].mask((df_grad['NACIO']==2)&(df_grad['NACIO1']!=1),1,inplace=True)
# Se imputa 0 (sin segunda nacionalidad) a quienes declararon solo nacionalidad española (1 en 'NACIO'):
df_grad['NACIO2'].mask((df_grad['NACIO']==1),0,inplace=True)
# Se imputa None al resto de no respuestas:
df_grad['NACIO2'].replace({' ':None},inplace=True)
df_grad['NACIO2']=df_grad['NACIO2'].astype('float')
df_grad[['NACIO','NACIO2']].groupby('NACIO').agg(['unique'])

,NACIO2
,unique
NACIO,
1,[0.0]
2,"[1.0, nan]"
3,"[nan, 1.0, 2.0]"


In [30]:
proporciones('NACIO2',dropna=False)

2.0    0.000253
1.0    0.008341
NaN    0.013270
0.0    0.978137
Name: NACIO2, dtype: float64

#### 'PAIS_NACI'
**País de nacimiento**: 1=España, 2=Otro país de la UE, 3=Otro país fuera de la UE, 9=NS/NC

In [31]:
df_grad['PAIS_NACI'].unique() # No missing values 

array([1, 3, 2])

#### Nueva variable 'eu27'
**Nacido en EU27**: 1=Nacido en EU27, 0=No nacido en EU27

In [32]:
df_grad['eu27']=0
df_grad['eu27'].mask((df_grad['PAIS_NACI']==1)|(df_grad['PAIS_NACI']==2),1,inplace=True)
df_grad[['eu27','PAIS_NACI']].groupby('eu27').agg(['unique']) # Comprobación

,PAIS_NACI
,unique
eu27,
0,[3]
1,"[1, 2]"


In [33]:
Variables.append('eu27')

#### 'TITU'
**Titulación**:  
011101=Pedagogía, 011201=Educación infantil, 011301=Educación primaria, 011401=Otros maestros, 011901=Educación social,   
021101=Audiovisual, imagen y multimedia, 021201=Diseño, 021301=Bellas artes, 021302=Historia del arte, 021401=Conservación y restauración, 021502=Música y Artes escénicas, 022201=Arqueología, 022202=Historia, 022301=Filosofía, 022901=Humanidades, 023101=Lengua inglesa, 023102=Lenguas clásicas, 023103=Otras lenguas extranjeras, 023104=Traducción e interpretación, 023201=Lenguas y dialectos españoles, 023202=Literatura, 023901=Lenguas modernas y aplicadas,   
031101=Economía, 031201=Política y gestión pública, 031202=Relaciones internacionales, 031301=Psicología, 031401=Antropología social y cultural y Estudios y gestión de la cultura, 031402=Criminología, 031404=Geografía, 031406=Sociología e Igualdad de género, 032101=Comunicación, 032102=Periodismo, 032201=Información y documentación,   
041201=Financiera y actuarial, 041202=Finanzas y contabilidad, 041301=Administración y empresa, 041302=Ciencias del trabajo, 041303=Gestión y administración pública, 041401=Marketing, 041402=Protocolo y eventos, 041403=Publicidad y relaciones públicas, 041601=Comercio, 042101=Derecho,   
051101=Biología, 051201=Bioquímica, 051202=Biotecnología, 051901=Biomedicina, 052101=Ciencias ambientales, 053101=Química, 053201=Ciencias del mar, 053202=Geografía y ordenación del territorio, 053203=Geología, 053301=Física, 054101=Matemáticas, 054201=Estadística,   
061301=Desarrollo de software y de aplicaciones e Ingeniería multimedia, 061901=Informática,   
071101=Ingeniería química industrial e Ingeniería medioambiental, 071301=Ingeniería de la energía, 071302=Ingeniería eléctrica, 071401=Ingeniería de computadores, 071402=Ingeniería de sonido e imagen, 071403=Ingeniería de telecomunicación, 071404=Ingeniería electrónica industrial y automática, 071405=Ingeniería en electrónica, 071501=Ingeniería en diseño industrial y desarrollo del producto, 071502=Ingeniería en tecnologías industriales, 071503=Ingeniería mecánica, 071601=Ingeniería aeronáutica, 071603=Ingeniería naval y oceánica, 071901=Ingeniería de organización industrial y Nanotecnología, 072101=Ciencia y tecnología de los alimentos e Ingeniería alimentaria, 072102=Enología, 072201=Ingeniería de materiales e Ingeniería textil, 072401	=Ingeniería de minas y energía, 073101=Arquitectura y Urbanismo y paisajismo, 073102=Ingeniería geomática, topografía y cartografía, 073201=Arquitectura técnica, 073202=Ingeniería civil,   
081102=Ingeniería agraria y agroalimentaria, 081103=Ingeniería agrícola, agropecuaria y medio rural, 081201=Ingeniería horticultura y jardinería, 082101=Ingeniería forestal y montes, 084101=Veterinaria,   
091101=Odontología, 091201=Medicina, 091301=Enfermería, 091401=Ingeniería biomédica y de la salud, 091402=Óptica y optometría, 091501=Fisioterapia, 091502=Logopedia, 091503=Nutrición humana y dietética, 091504=Podología, 091505=Terapia ocupacional, 091601=Farmacia, 092301=Trabajo social,   
101401=Actividad física y del deporte, 101501=Turismo, 104101=Náutica y transporte marítimo, 104103=Servicio de transporte terrestre y Servicio de transporte aéreo, 109999=Servicios (otros estudios).

In [34]:
df_grad['TITU'].unique() # No missing values

array([ 11101,  11201,  11301,  11401,  11901,  21101,  21201,  21301,
        21302,  21401,  21502,  22201,  22202,  22301,  22901,  23101,
        23102,  23103,  23104,  23201,  23202,  23901,  31101,  31201,
        31202,  31301,  31401,  31402,  31404,  31406,  32101,  32102,
        32201,  41201,  41202,  41301,  41302,  41303,  41401,  41402,
        41403,  41601,  42101,  51101,  51201,  51202,  51901,  52101,
        53101,  53201,  53202,  53203,  53301,  54101,  54201,  61301,
        61901,  71101,  71301,  71302,  71401,  71402,  71403,  71404,
        71405,  71501,  71502,  71503,  71601,  71603,  71901,  72101,
        72102,  72201,  72401,  73101,  73102,  73201,  73202,  81102,
        81103,  81201,  82101,  84101,  91101,  91201,  91301,  91401,
        91402,  91501,  91502,  91503,  91504,  91505,  91601,  92301,
       101401, 101501, 104101, 104103, 109999])

#### 'RAMA' (redesignada como 'rama')
**Rama de conocimiento** de la titulación:  
1=Artes y humanidades, 2=Ciencias, 3=Ciencias sociales y jurídicas, 4=Ingeniería y arquitectura, 5=Ciencias de la salud

In [35]:
df_grad['RAMA'].unique() # No missing values

array([3, 1, 4, 5, 2])

In [36]:
df_grad.rename(columns={'RAMA':'rama'},inplace=True)

In [37]:
proporciones('rama')

2    0.087738
1    0.100376
5    0.141796
4    0.211842
3    0.458248
Name: rama, dtype: float64

In [38]:
# Varias titulaciones adscritas a más de una rama del conocimiento
df_grad[['rama','TITU']].groupby('TITU').agg(['unique'])

,rama
,unique
TITU,
11101,"[3, 1]"
11201,[3]
11301,[3]
11401,[3]
11901,[3]
...,...
101401,"[3, 5]"
101501,[3]


In [39]:
df_grad[['rama','TITU']].groupby('rama').agg(['unique'])

,TITU
,unique
rama,
1,"[11101, 21101, 21201, 21301, 21302, 21401, 21502, 22201, 22202, 22301, 22901, 23101, 23102, 23103, 23104, 23201, 23202, 23901, 31401, 31404, 32101, 32102, 32201, 53202, 73101]"
2,"[51101, 51201, 51202, 51901, 52101, 53101, 53201, 53203, 53301, 54101, 54201, 71901, 72101, 72102, 91402, 91503]"
3,"[11101, 11201, 11301, 11401, 11901, 21101, 23104, 31101, 31201, 31202, 31301, 31401, 31402, 31404, 31406, 32101, 32102, 32201, 41201, 41202, 41301, 41302, 41303, 41401, 41402, 41403, 41601, 42101, 53202, 54201, 91501, 92301, 101401, 101501, 104103, 109999]"
4,"[21101, 21201, 53203, 54101, 61301, 61901, 71101, 71301, 71302, 71401, 71402, 71403, 71404, 71405, 71501, 71502, 71503, 71601, 71603, 71901, 72101, 72201, 72401, 73101, 73102, 73201, 73202, 81102, 81103, 81201, 82101, 91401, 104101, 104103, 109999]"
5,"[31301, 51101, 51901, 72101, 84101, 91101, 91201, 91301, 91402, 91501, 91502, 91503, 91504, 91505, 91601, 101401]"


In [40]:
# Distribucion de sexos condicionada por ramas de conocimiento: 
dist_condicionada('rama','fem',rev=True)
# Solo la rama ciencias tiene una distribución de sexos similar a la general entre los graduados. 
# La mayor presencia femenina se da en ciencias de la salud. La menor, en ingeniería y arquitectura. 

fem
rama fem          
1    0    0.399433
     1    0.600567
2    0    0.452647
     1    0.547353
3    0    0.359625
     1    0.640375
4    0    0.675168
     1    0.324832
5    0    0.295455
     1    0.704545

In [41]:
# Distribucion de nacionalidad española y otras condicionada por ramas de conocimiento: 
dist_condicionada('rama','esp',rev=True)
# Artes y humanidades y ciencias de la salud son las ramas de conocimiento en las que la presencia de ...
# ... nacionalidades no españolas es superior a la general entre los graduados. 

esp
rama esp          
1    0    0.018571
     1    0.981429
2    0    0.007922
     1    0.992078
3    0    0.010549
     1    0.989451
4    0    0.007606
     1    0.992394
5    0    0.014260
     1    0.985740

#### Nuevas variables dicotómicas: 'art_hum', 'ciencias', 'cc_soc_ju', 'ing_arq', 'cc_salud'
'art_hum': 1=Rama artes y humanidades, 0=Otra rama,  
'ciencias': 1=Rama ciencias, 0=Otra rama,  
'cc_soc_ju': 1=Rama ciencias sociales y jurídicas, 0=Otra rama,  
'ing_arq': 1=Rama ingeniería y arquitectura, 0=Otra rama,  
'cc_salud': 1=Rama cc_salud, 0=Otra rama,

In [42]:
L_ramas=['art_hum','ciencias','cc_soc_ju','ing_arq','cc_salud']
for i,rama in enumerate (L_ramas): 
    df_grad[rama]=0
    df_grad[rama].mask(df_grad['rama']==i+1,1,inplace=True)
df_grad[L_ramas+['rama']].groupby('rama').agg(['unique']) # Comprobación: 

,art_hum,ciencias,cc_soc_ju,ing_arq,cc_salud
,unique,unique,unique,unique,unique
rama,,,,,
1,[1],[0],[0],[0],[0]
2,[0],[1],[0],[0],[0]
3,[0],[0],[1],[0],[0]
4,[0],[0],[0],[1],[0]
5,[0],[0],[0],[0],[1]


In [43]:
for vble in L_ramas: 
    Variables.append(vble)

#### 'AMBITO'
**Ámbito de estudio**: 0112=Formación de docentes de enseñanza infantil, 0113=Formación de docentes de enseñanza primaria, 011=Educación (Otros estudios), 0211=Técnicas audiovisuales y medios de comunicación, 021=Artes (Otros estudios), 022=Humanidades, 023=Lenguas, 0311=Economía, 0313=Psicología, 031=Ciencias sociales y del comportamiento (Otros estudios), 032=Periodismo y documentación, 0413=Dirección y administración, 041=Negocios y administración (Otros estudios), 042=Derecho, 051=Ciencias de la vida, 052=Medio ambiente, 053=Ciencias químicas, físicas y geológicas, 054=Matemáticas y estadística, 061=Informática, 071=Ingeniería y profesiones afines, 072=Industria manufacturera y producción, 073=Arquitectura y construcción, 081=Agricultura y ganadería, 082=Silvicultura, 084=Veterinaria, 0912=Medicina, 0913=Enfermería, 091=Salud (Otros estudios), 092=Servicios sociales, 1014=Actividades físicas y deportivas, 1015=Viajes, turismo y ocio, 109=Servicios (Otros estudios), 9999=NS/NC.

In [44]:
df_grad['AMBITO'].unique() # No missing values

array([  11,  112,  113,  211,   21,   22,   23,  311,   31,  313,   32,
         41,  413,   42,   51,   52,   53,   54,   61,   71,   72,   73,
         81,   82,   84,   91,  912,  913,   92, 1014, 1015,  109])

In [45]:
df_grad['AMBITO'].dtype

dtype('int64')

In [46]:
# Todas las titulaciones adscritas a un único ámbito de estudio.  
for row in range(0,len(df_grad[['TITU','AMBITO']].groupby('TITU'))):
    print (df_grad[['TITU','AMBITO']].groupby('TITU').agg(['unique']).iloc[row])

AMBITO  unique    [11]
Name: 11101, dtype: object
AMBITO  unique    [112]
Name: 11201, dtype: object
AMBITO  unique    [113]
Name: 11301, dtype: object
AMBITO  unique    [11]
Name: 11401, dtype: object
AMBITO  unique    [11]
Name: 11901, dtype: object
AMBITO  unique    [211]
Name: 21101, dtype: object
AMBITO  unique    [21]
Name: 21201, dtype: object
AMBITO  unique    [21]
Name: 21301, dtype: object
AMBITO  unique    [21]
Name: 21302, dtype: object
AMBITO  unique    [21]
Name: 21401, dtype: object
AMBITO  unique    [21]
Name: 21502, dtype: object
AMBITO  unique    [22]
Name: 22201, dtype: object
AMBITO  unique    [22]
Name: 22202, dtype: object
AMBITO  unique    [22]
Name: 22301, dtype: object
AMBITO  unique    [22]
Name: 22901, dtype: object
AMBITO  unique    [23]
Name: 23101, dtype: object
AMBITO  unique    [23]
Name: 23102, dtype: object
AMBITO  unique    [23]
Name: 23103, dtype: object
AMBITO  unique    [23]
Name: 23104, dtype: object
AMBITO  unique    [23]
Name: 23201, dtype: obje

In [47]:
# 'AMBITO' determinado por 2, 3 o 4 primeros dígitos de 'TITU' excepto los siguientes casos: 
cond1=(df_grad['AMBITO'].astype('str')!=df_grad['TITU'].astype('str').str[:2])
cond2=(df_grad['AMBITO'].astype('str')!=df_grad['TITU'].astype('str').str[:3])
cond3=(df_grad['AMBITO'].astype('str')!=df_grad['TITU'].astype('str').str[:4])
df_grad[['AMBITO','TITU','rama']][cond1&cond2&cond3].value_counts()

AMBITO  TITU    rama
109     104101  4       109
        104103  3        47
                4        10
dtype: int64

In [48]:
# Varios ámbitos de estudio asociados a más de una rama de conocimiento: 
df_grad[['rama','AMBITO']].groupby('AMBITO').agg(['unique'])

,rama
,unique
AMBITO,
11,"[3, 1]"
21,"[1, 4]"
22,[1]
23,"[1, 3]"
31,"[3, 1]"
32,"[3, 1]"
41,[3]
42,[3]


#### 'T_UNIV'
**Tipo de universidad**: 1=Universidad Pública presencial, 2=Universidad Pública a distancia, 3=Universidad Privada presencial, 4=Universidad Privada a distancia.

In [49]:
df_grad['T_UNIV'].unique() # No missing values

array([2, 1, 4, 3])

In [50]:
proporciones('T_UNIV')

4    0.027803
2    0.034122
3    0.119838
1    0.818236
Name: T_UNIV, dtype: float64

#### Nueva variable: 'tipo_centro'
**Tipo de centro educativo**: 1=Centros públicos, 0=Centros privados  

Variable proxi (en otros datasets las respuestas están referidas a niveles educativos diferentes).

In [51]:
df_grad['tipo_centro']=0
df_grad['tipo_centro'].mask((df_grad['T_UNIV']==1)|(df_grad['T_UNIV']==2),1,inplace=True)
df_grad[['T_UNIV','tipo_centro']].groupby('tipo_centro').agg(['unique']) # Comprobación: 

,T_UNIV
,unique
tipo_centro,
0,"[4, 3]"
1,"[2, 1]"


In [52]:
Variables.append('tipo_centro')

In [53]:
proporciones('tipo_centro')

0    0.147641
1    0.852359
Name: tipo_centro, dtype: float64

In [54]:
# Distribucion de centros públicos y privados condicionada por nacionalidad española u otras: 
dist_condicionada('esp','tipo_centro',rev=True)
# La mayor frecuencia de elección de centros públicos en general se acentúa en el caso de aquellos cuya ...
# ... nacionalidad no es española. 

tipo_centro
esp tipo_centro             
0   0               0.097421
    1               0.902579
1   0               0.148201
    1               0.851799

#### Nueva variable: 'a_dist'
**Formación presencial o a distancia**: 1=Formación a distancia, 0=Formación presencial  


In [55]:
df_grad['a_dist']=0
df_grad['a_dist'].mask((df_grad['T_UNIV']==2)|(df_grad['T_UNIV']==4),1,inplace=True)
df_grad[['T_UNIV','a_dist']].groupby('a_dist').agg(['unique']) # Comprobación: 

,T_UNIV
,unique
a_dist,
0,"[1, 3]"
1,"[2, 4]"


In [56]:
Variables.append('a_dist')

In [57]:
proporciones('a_dist')

1    0.061925
0    0.938075
Name: a_dist, dtype: float64

#### 'DISCA'
Tiene **discapacidad reconocida** superior al 33 %: 1=Sí, 2=No.  

Nota: Se modifica valor No: 0=No. 

In [58]:
df_grad['DISCA'].unique() # No missing values

array([2, 1])

In [59]:
# Cambio del valor asociado a respuesta No: 
df_grad['DISCA'].replace({2:0},inplace=True)

In [60]:
proporciones('DISCA')

1    0.012196
0    0.987804
Name: DISCA, dtype: float64

In [61]:
# Discapacidad según tipo de universidad (en términos absolutos)
df_grad[['T_UNIV','DISCA']].groupby('DISCA').agg(['value_counts']) 

T_UNIV
             value_counts
DISCA T_UNIV             
0     1             25583
      3              3770
      2              1051
      4               861
1     1               315
      2                29
      3                23
      4                19

In [62]:
# Proporción de discapacidad según el tipo de centro: 
dist_condicionada('DISCA','T_UNIV')
# La proporción de estudiantes con discapacidad en las universidades a distancia públicas y privadas ...
# ... es al menos el doble que en las presenciales públicas y privadas respectivamente

DISCA
T_UNIV DISCA          
1      0      0.987837
       1      0.012163
2      0      0.973148
       1      0.026852
3      0      0.993936
       1      0.006064
4      0      0.978409
       1      0.021591

In [63]:
# Proporción de cada tipo de universidad según discapacidad: 
dist_condicionada('DISCA','T_UNIV',rev=True)
# Los estudiantes con y sin discapacidad eligen universidad pública presencial mayoritariamente y ... 
# ... aproximadamente con la misma frecuencia. 
# Pero los estudiantes con discapacidad eligen la universidad a distancia (pública o privada) con más del doble ...
# ... de frecuencia que los estudiantes sin discapacidad. La relación se invierte para la privada presencial.

T_UNIV
DISCA T_UNIV          
0     1       0.818263
      2       0.033616
      3       0.120582
      4       0.027539
1     1       0.816062
      2       0.075130
      3       0.059585
      4       0.049223

#### 'PROXY'
**Tipo de entrevista**: 1=Entrevista directa con la persona seleccionada, 2=Entrevista proxy.

In [64]:
df_grad['PROXY'].unique() # No missing values

array([1, 2])

In [65]:
proporciones('PROXY')

2    0.012353
1    0.987647
Name: PROXY, dtype: float64

#### 'LUG_RES_PAIS'
**País de residencia actual**: 01=España,02=Reino Unido, 03=Alemania, 04=Francia, 05=Otro país de la Unión Europea, 06=Otro país europeo (fuera de la UE), 07=Un país de América del Norte, 08=Un país de América Central o América del Sur, 09=Un país de Asia, 10=Un país de África, 11=Un país de Oceanía, 99=NS/NC.

In [66]:
df_grad['LUG_RES_PAIS'].sort_values().unique() # No missing values

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [67]:
proporciones('LUG_RES_PAIS')

10    0.000379
11    0.001517
9     0.002117
8     0.002591
6     0.003791
4     0.004486
7     0.004644
3     0.006793
5     0.013586
2     0.015987
1     0.944109
Name: LUG_RES_PAIS, dtype: float64

#### 'PAIS_NAC_PADRE'
**País de nacimiento del padre**: 1=España, 2=Otro país de la UE, 3=Otro país fuera de la UE, 9=NS/NC.

In [68]:
df_grad['PAIS_NAC_PADRE'].unique() # Missing values as 9

array([1, 9, 2, 3])

In [69]:
# Se imputa None a NS/NC (9):
df_grad['PAIS_NAC_PADRE'].replace({9:None},inplace=True)

In [70]:
proporciones('PAIS_NAC_PADRE')
# Frecuencia de padre no español entre los graduados inferior a la observada en la población general (aprox. 10 %)

2    0.011928
3    0.027037
1    0.961034
Name: PAIS_NAC_PADRE, dtype: float64

#### 'ESTUDIOS_PADRE'
**Nivel de formación máximo alcanzado por el padre**: 1=No sabe leer o escribir, 2=Educación primaria incompleta (ha asistido menos de 5 años a la escuela), 3=Educación primaria completa, 4=Primera etapa de Enseñanza Secundaria, con o sin título (ESO, EGB, Bachillerato Elemental), 5=Estudios de Bachillerato (Bachillerato LOGSE, BUP, COU, Preu), 6=Enseñanzas profesionales de grado medio o equivalentes, 7=Enseñanzas profesionales de grado superior o equivalentes, 8=Estudios universitarios (Diplomatura, Licenciatura, Doctorado) o equivalentes, 9=NS/NC.

In [71]:
df_grad['ESTUDIOS_PADRE'].unique() # Missing values as 9

array([2, 3, 7, 8, 5, 4, 6, 9, 1])

In [72]:
# Se imputa None a NS/NC (9):
df_grad['ESTUDIOS_PADRE'].replace({9:None},inplace=True)

In [73]:
proporciones('ESTUDIOS_PADRE',dropna=False)

1.0    0.003570
2.0    0.052921
NaN    0.069508
7.0    0.073078
6.0    0.076111
5.0    0.114025
4.0    0.159458
3.0    0.166219
8.0    0.285109
Name: ESTUDIOS_PADRE, dtype: float64

#### Nueva variable: 'univ_pa'
**Padre** con o sin **estudios universitarios**: 1=Con estudios universitarios, 0=Sin estudios universitarios 

In [74]:
df_grad['univ_pa']=0
df_grad['univ_pa'].mask(df_grad['ESTUDIOS_PADRE']==8,1,inplace=True)
df_grad[['ESTUDIOS_PADRE','univ_pa']].groupby('univ_pa').agg(['unique']) # Comprobación: 

,ESTUDIOS_PADRE
,unique
univ_pa,
0,"[2, 3, 7, 5, 4, 6, None, 1]"
1,[8]


In [75]:
Variables.append('univ_pa')

In [76]:
# Distribucion de padre universitario o no según nacionalidad española u otras: 
dist_condicionada('univ_pa','esp')
# La frecuencia de padre universitario es mayor entre los graduados con nacionalidad no española.

univ_pa
esp univ_pa          
0   0        0.673352
    1        0.326648
1   0        0.715354
    1        0.284646

In [77]:
# Distribucion de padre universitario o no según nacidos o no en EU27: 
dist_condicionada('univ_pa','eu27')
# La frecuencia de padre universitario es aun mayor entre los no nacidos en EU27. Parece reflejar mayores ...
# ... barreras de acceso a la universidad española para los estudiantes no europeos. 

univ_pa
eu27 univ_pa          
0    0        0.628247
     1        0.371753
1    0        0.716610
     1        0.283390

#### 'PAIS_NAC_MADRE'
**País de nacimiento de la madre**: 1=España, 2=Otro país de la UE, 3=Otro país fuera de la UE, 9=NS/NC.

In [78]:
df_grad['PAIS_NAC_MADRE'].unique() # Missing values as 9

array([1, 3, 9, 2])

In [79]:
# Se imputa None a NS/NC (9):
df_grad['PAIS_NAC_MADRE'].replace({9:None},inplace=True)

In [80]:
proporciones('PAIS_NAC_MADRE')

2    0.014427
3    0.026232
1    0.959341
Name: PAIS_NAC_MADRE, dtype: float64

#### 'ESTUDIOS_MADRE'
**Nivel de formación máximo alcanzado por la madre**: 1=No sabe leer o escribir, 2=Educación primaria incompleta (ha asistido menos de 5 años a la escuela), 3=Educación primaria completa, 4=Primera etapa de Enseñanza Secundaria, con o sin título (ESO, EGB, Bachillerato Elemental), 5=Estudios de Bachillerato (Bachillerato LOGSE, BUP, COU, Preu), 6=Enseñanzas profesionales de grado medio o equivalentes, 7=Enseñanzas profesionales de grado superior o equivalentes, 8=Estudios universitarios (Diplomatura, Licenciatura, Doctorado) o equivalentes, 9=NS/NC.

In [81]:
df_grad['ESTUDIOS_MADRE'].unique() # Missing values as 9

array([3, 7, 2, 4, 5, 6, 8, 9, 1])

In [82]:
# Se imputa None a NS/NC (9):
df_grad['ESTUDIOS_MADRE'].replace({9:None},inplace=True)

In [83]:
proporciones('ESTUDIOS_MADRE')

1    0.004141
2    0.053836
7    0.062053
6    0.095945
5    0.129439
3    0.175557
4    0.192784
8    0.286244
Name: ESTUDIOS_MADRE, dtype: float64

#### Nueva variable: 'univ_ma'
**Madre** con o sin **estudios universitarios**: 1=Con estudios universitarios, 0=Sin estudios universitarios 

In [84]:
df_grad['univ_ma']=0
df_grad['univ_ma'].mask(df_grad['ESTUDIOS_MADRE']==8,1,inplace=True)
df_grad[['ESTUDIOS_MADRE','univ_ma']].groupby('univ_ma').agg(['unique']) # Comprobación: 

,ESTUDIOS_MADRE
,unique
univ_ma,
0,"[3, 7, 2, 4, 5, 6, None, 1]"
1,[8]


In [85]:
Variables.append('univ_ma')

In [86]:
# Distribucion de madre universitaria o no según nacionalidad española u otras: 
dist_condicionada('univ_ma','esp')
# La frecuencia de madre universitaria es mayor entre los graduados con nacionalidad no española.

univ_ma
esp univ_ma          
0   0        0.670487
    1        0.329513
1   0        0.727653
    1        0.272347

In [87]:
# Distribucion de madre universitaria o no según nacidos o no en EU27: 
dist_condicionada('univ_ma','eu27')
# De nuevo, la frecuencia de madre universitaria es aun mayor entre los no nacidos en EU27, aunque con menor ...
# ... diferencia que en el caso de los padres. 

univ_ma
eu27 univ_ma          
0    0        0.660714
     1        0.339286
1    0        0.728339
     1        0.271661

#### 'EST_B1' (redesignada como 'beca')
¿Disfrutó de **alguna beca** durante sus estudios?: 1=Sí, 2=No, 9=NS/NC.  

Nota: se modifica el valor asociado a No: 0=No

In [88]:
df_grad['EST_B1'].unique() # Missing values as 9

array([2, 1, 9])

In [89]:
# Redefinición de la variable 'EST_B1' como 'beca':
df_grad.rename(columns={'EST_B1':'beca'},inplace=True)
# Se imputa None a NS/NC (9):
df_grad['beca'].replace({9:None},inplace=True)
# Se modifica el valor asociado a respuesta No: 
df_grad['beca'].replace({2:0},inplace=True)

In [90]:
Variables.append('beca')

In [91]:
proporciones('beca')

1.0    0.466688
0.0    0.533312
Name: beca, dtype: float64

In [92]:
# Distribución de becas condicionadas por el sexo: 
dist_condicionada('beca','fem')
# La proporción de becas de estudios es superior entre las mujeres que entre los hombres. 

beca
fem beca          
0   0.0   0.568505
    1.0   0.431495
1   0.0   0.506817
    1.0   0.493183

In [93]:
# Distribucion de becas condicionadas por nacionalidad española u otras: 
dist_condicionada('beca','esp')
# La proporción de becas de estudios es superior entre aquellos cuya nacionalidad es no española

beca
esp beca          
0   0.0   0.363897
    1.0   0.636103
1   0.0   0.535203
    1.0   0.464797

In [94]:
# Distribucion de becas condicionadas por continente de nacimento:  
dist_condicionada('beca','eu27')
# La proporción de becas de estudios es superior entre los no nacidos en EU27. 

beca
eu27 beca          
0    0.0   0.331169
     1.0   0.668831
1    0.0   0.537328
     1.0   0.462672

#### 'EST_B2_1'
(Solo para respuesta Sí (1) en variable 'beca')  

¿Disfrutó de una **beca general para el estudio**?: 1=Sí, 2=No, 9=NS/NC.

Nota: se modifica el valor asociado a No: 0=No

In [95]:
df_grad['EST_B2_1'].unique() # Missing values as ' ' or '9'

array([' ', '1', '2', '9'], dtype=object)

In [96]:
# Función re_EST_B1: inspeccion e imputación de valores de variables condicionadas por respuesta a 'beca'.
def re_beca(col):
    print (df_grad[['beca',col]].groupby('beca').agg(['unique']))
    # Se imputa respuesta no (0) a quienes manifestaron no haber disfrutado de beca (0 en 'EST_B1')
    df_grad[col].mask(df_grad['beca']==0,0,inplace=True)
    # Se imputa None a respuestas en blanco y NS/NC y se cambia el valor No (2) por 0:
    df_grad[col].replace({' ':None,'9':None,'2':0},inplace=True)
    df_grad[col]=df_grad[col].astype('float')
    print (df_grad[['beca',col]].groupby('beca').agg(['unique']))
    return

In [97]:
re_beca('EST_B2_1')

       EST_B2_1
         unique
beca           
0.0         [ ]
1.0   [1, 2, 9]
             EST_B2_1
               unique
beca                 
0.0             [0.0]
1.0   [1.0, 0.0, nan]


In [98]:
proporciones('EST_B2_1')

1.0    0.383572
0.0    0.616428
Name: EST_B2_1, dtype: float64

#### 'EST_B2_2'
(Solo para respuesta Sí (1) en variable 'beca')  

¿Disfrutó de algún **premio o beca de excelencia**?: 1=Sí, 2=No, 9=NS/NC.  

Nota: se modifica el valor asociado a No: 0=No.

In [99]:
df_grad['EST_B2_2'].unique() # Missing values as ' ' or '9'

array([' ', '2', '1', '9'], dtype=object)

In [100]:
re_beca('EST_B2_2')

       EST_B2_2
         unique
beca           
0.0         [ ]
1.0   [2, 1, 9]
             EST_B2_2
               unique
beca                 
0.0             [0.0]
1.0   [0.0, 1.0, nan]


In [101]:
proporciones('EST_B2_2')

1.0    0.044373
0.0    0.955627
Name: EST_B2_2, dtype: float64

#### 'EST_B2_3'
(Solo para respuesta Sí (1) en variable 'beca')  

¿Disfrutó de alguna **beca de colaboración** en la universidad?: 1=Sí, 2=No, 9=NS/NC.  

Nota: se modifica el valor asociado a No: 0=No.

In [102]:
df_grad['EST_B2_3'].unique() # Missing values as ' ' or '9'

array([' ', '2', '1', '9'], dtype=object)

In [103]:
re_beca('EST_B2_3')

       EST_B2_3
         unique
beca           
0.0         [ ]
1.0   [2, 1, 9]
             EST_B2_3
               unique
beca                 
0.0             [0.0]
1.0   [0.0, 1.0, nan]


In [104]:
proporciones('EST_B2_3')

1.0    0.035479
0.0    0.964521
Name: EST_B2_3, dtype: float64

#### 'EST_B2_4'
(Solo para respuesta Sí (1) en variable 'beca')  

¿Disfrutó de alguna **beca de prácticas externas** o para prestar un servicio o desempeñar un trabajo fuera de la Universidad?: 1=Sí, 2=No, 9=NS/NC.  


Nota: se modifica el valor asociado a No: 0=No.

In [105]:
df_grad['EST_B2_4'].unique() # Missing values as ' ' or '9'

array([' ', '2', '1', '9'], dtype=object)

In [106]:
re_beca('EST_B2_4')

       EST_B2_4
         unique
beca           
0.0         [ ]
1.0   [2, 1, 9]
             EST_B2_4
               unique
beca                 
0.0             [0.0]
1.0   [0.0, 1.0, nan]


In [107]:
proporciones('EST_B2_4')

1.0    0.037893
0.0    0.962107
Name: EST_B2_4, dtype: float64

#### 'EST_B2_5'
(Solo para respuesta Sí (1) en variable 'beca')  

¿Disfrutó de alguna **beca para realizar estudios fuera de España** (incluidas becas complemento a programas Erasmus)?: 1=Sí, 2=No, 9=NS/NC.  

Nota: se modifica el valor asociado a No: 0=No.

In [108]:
df_grad['EST_B2_5'].unique() # Missing values as ' ' or '9'

array([' ', '2', '1', '9'], dtype=object)

In [109]:
re_beca('EST_B2_5')

       EST_B2_5
         unique
beca           
0.0         [ ]
1.0   [2, 1, 9]
             EST_B2_5
               unique
beca                 
0.0             [0.0]
1.0   [0.0, 1.0, nan]


In [110]:
proporciones('EST_B2_5')

1.0    0.106216
0.0    0.893784
Name: EST_B2_5, dtype: float64

#### 'EST_M1' (redesignada como 'extr')
(Solo para respuesta distinta de Sí (1) en variable 'EST_B2_5')  

¿Realizó parte de sus **estudios en el extranjero**?: 1=Sí, 2=No, 9=NS/NC.  

Nota: se modifica el valor asociado a No: 0=No.

In [111]:
df_grad['EST_M1'].unique() # Missing values as '9'

array([2, 1, 9])

In [112]:
df_grad.rename(columns={'EST_M1':'extr'},inplace=True)

In [113]:
df_grad[['EST_B2_5','extr']].groupby('EST_B2_5').agg(['unique'])
# Imputada por el encuestador respuesta Si (1) a quienes manifestaron haber disfrutado de beca para ...
# ... estudiar fuera de España ('EST_B2_5')

,extr
,unique
EST_B2_5,
0.0,"[2, 1, 9]"
1.0,[1]


In [114]:
# Se imputa None a NS/NC (9) y se cambia el valor asociado a No (2) por 0: 
df_grad['extr'].replace({9:None,2:0},inplace=True)
df_grad[['EST_B2_5','extr']].groupby('EST_B2_5').agg(['unique']) # Comprobacion: 

,extr
,unique
EST_B2_5,
0.0,"[0.0, 1.0, nan]"
1.0,[1.0]


In [115]:
Variables.append('extr')

In [116]:
proporciones('extr')

1.0    0.167273
0.0    0.832727
Name: extr, dtype: float64

In [117]:
# Proporciones de estudios en el extranjero con y sin beca específica: 
df_grad['EST_B2_5'][df_grad['extr']==1].value_counts(normalize=True)

1.0    0.634054
0.0    0.365946
Name: EST_B2_5, dtype: float64

#### Nueva variable: 'extr_sin_beca'
¿Realizó parte de sus **estudios en el extranjero sin beca** específica?: 1=Si, 0=No.  

Variable proxi de renta familiar alta.  

In [118]:
df_grad['extr_sin_beca']=0
df_grad['extr_sin_beca'].mask((df_grad['extr']==1)&(df_grad['EST_B2_5']!=1),1,inplace=True)
df_grad[['extr_sin_beca','extr','EST_B2_5']].groupby(['extr','EST_B2_5']).agg(['unique']) # Comprobación

extr_sin_beca
                     unique
extr EST_B2_5              
0.0  0.0                [0]
1.0  0.0                [1]
     1.0                [0]

In [119]:
Variables.append('estr_sin_beca')

#### 'EST_M2'
(Solo para respuesta Sí (1) en variable 'EST_B2_5')  

**Programa o beca de movilidad**: 1=Programa Erasmus, 2=Otros programas o becas dentro de la UE, 3=Otros programas o becas fuera de la UE, 9=NS/NC.  

Se añade respuesta 0=Ninguna. 

In [120]:
df_grad['EST_M2'].unique() # Missing values as ' ' or '9'

array([' ', '3', '1', '2', '9'], dtype=object)

In [121]:
df_grad[['EST_B2_5','EST_M2']].groupby('EST_B2_5',dropna=False).agg(['unique'])

,EST_M2
,unique
EST_B2_5,
0.0,[ ]
1.0,"[3, 1, 2, 9]"
NaN,[ ]


In [122]:
# Se imputa valor 0 ('Ninguna') a quienes manifestaron no haber disfrutado de beca para estudiar fuera ...
# ... de España (respuesta 2 en 'EST_B2_5').
df_grad['EST_M2'].mask(df_grad['EST_B2_5']==0,0,inplace=True)
# Se imputa None a NS/NC (9) y a respuesta en blanco: 
df_grad['EST_M2'].replace({'9':None,' ':None},inplace=True)
df_grad['EST_M2']=df_grad['EST_M2'].astype('float')
df_grad[['EST_B2_5','EST_M2']].groupby('EST_B2_5',dropna=False).agg(['unique'])

,EST_M2
,unique
EST_B2_5,
0.0,[0.0]
1.0,"[3.0, 1.0, 2.0, nan]"
NaN,[nan]


In [123]:
proporciones('EST_M2',dropna=False)

NaN    0.005339
2.0    0.005339
3.0    0.008594
1.0    0.091687
0.0    0.889040
Name: EST_M2, dtype: float64

In [124]:
# Distribución de tipos de becas de estudios en el extranjero entre quienes manifestaron haber disfrutado alguna: 
df_grad['EST_M2'][df_grad['EST_B2_5']==1].value_counts(normalize=True,ascending=True)

2.0    0.050553
3.0    0.081364
1.0    0.868083
Name: EST_M2, dtype: float64

#### 'EST_M3'
(Solo para respuesta Sí (1) en variable 'extr') 

**País de destino** cuando estudió fuera de España: 01=Reino Unido, 02=Alemania, 03=Francia, 04=Italia, 05=Portugal, 06=Países Nórdicos (Suecia, Noruega, Finlandia, Dinamarca e Islandia), 07=Otro país de la Unión Europea, 08=Otro país europeo, 11=Un país de América del Norte, 12=Un país de América Central o América del Sur, 21=Un país de Asia, 31=Un país de África, 41=Un país de Oceanía, 99=NS/NC.  

Nota: se añade valor 0=Ninguno. 

In [125]:
df_grad['EST_M3'].sort_values().unique() # Missing values as '  ' or '99'

array(['  ', '01', '02', '03', '04', '05', '06', '07', '08', '11', '12',
       '21', '31', '41', '99'], dtype=object)

In [126]:
df_grad[['extr','EST_M3']].groupby('extr',dropna=False).agg(['unique'])

,EST_M3
,unique
extr,
0.0,[ ]
1.0,"[12, 03, 08, 05, 04, 02, 11, 06, 01, 07, 21, 99, 31, 41]"
NaN,[ ]


In [127]:
# Se imputa 0 ('Ninguno') a quienes no estudiaron fuera de España: 
df_grad['EST_M3'].mask(df_grad['extr']==0,0,inplace=True)
# Se imputa None a respuestas en blanco y NS/NC: 
df_grad['EST_M3'].replace({'  ':None,'99':None},inplace=True)
df_grad[['extr','EST_M3']].groupby('extr',dropna=False).agg(['unique'])

,EST_M3
,unique
extr,
0.0,[0]
1.0,"[12, 03, 08, 05, 04, 02, 11, 06, 01, 07, 21, None, 31, 41]"
NaN,[None]


In [128]:
# Distribución de países extranjeros donde se realiza una parte de los estudios entre quienes han estudiado fuera: 
df_grad['EST_M3'][df_grad['EST_M3']!=0].value_counts(normalize=True,ascending=True)
# Italia y Reino Unido son los destinos más frecuentes (excluyendo la categoría 'Otros países de la UE')

31    0.002470
41    0.004180
21    0.015200
08    0.047691
11    0.049211
05    0.054722
12    0.057192
06    0.076572
02    0.088543
03    0.089303
01    0.127874
04    0.172715
07    0.214326
Name: EST_M3, dtype: float64

#### 'EST_M4'
(Solo para respuesta Sí (1) en variable 'extr'-)  

**Duración** en meses del periodo en el que estudió fuera de España: 1=De 1 a 3 meses, 2=De 4 a 6 meses, 3=De 7 a 9 meses, 4=De 10 a 12 meses, 5=Más de un año, 9=NS/NC.  

Se añade respuesta 0=No estudió en el extranjero.

In [129]:
df_grad['EST_M4'].unique() # Missing values as ' ' or '9'

array([' ', '1', '4', '3', '2', '5', '9'], dtype=object)

In [130]:
df_grad[['extr','EST_M4']].groupby('extr',dropna=False).agg(['unique'])

,EST_M4
,unique
extr,
0.0,[ ]
1.0,"[1, 4, 3, 2, 5, 9]"
NaN,[ ]


In [131]:
# Se imputa 0 a quienes no estudiaron fuera de España: 
df_grad['EST_M4'].mask(df_grad['extr']==0,0,inplace=True)
# Se imputa None a respuestas en blanco y NS/NC
df_grad['EST_M4'].replace({' ':None,'9':None},inplace=True)
df_grad['EST_M4']=df_grad['EST_M4'].astype('float')
df_grad[['extr','EST_M4']].groupby('extr',dropna=False).agg(['unique'])

,EST_M4
,unique
extr,
0.0,[0.0]
1.0,"[1.0, 4.0, 3.0, 2.0, 5.0, nan]"
NaN,[nan]


#### 'EST_MES' 
¿Realizó parte de sus estudios para obtener su título de grado en **otra universidad española**? 1=Sí, 2=No, 9=NS/NC.  

Nota: se modifica el valor asociado a la respuesta No: 0=No.

In [132]:
df_grad['EST_MES'].unique() # Missing values as '9'

array([2, 1, 9])

In [133]:
# Se imputa None a NS/NC y se modifica respuesa No como 0:
df_grad['EST_MES'].replace({9:None,2:0},inplace=True)
df_grad['EST_MES'].unique() 

array([ 0.,  1., nan])

In [134]:
proporciones('EST_MES')

1.0    0.096747
0.0    0.903253
Name: EST_MES, dtype: float64

#### 'EST_B8' (redesignada como 'motivacion')
**Motivo principal** por el que ha realizado sus **estudios** de grado: 1=Formarse pensando en su futuro laboral, 2=Ampliar conocimientos por satisfacción personal, 3=Otros motivos, 9=NS/NC.

In [135]:
df_grad['EST_B8'].unique() # Missing values as '9'

array([1, 2, 3, 9])

In [136]:
# Redefinición de la variable: 
df_grad.rename(columns={'EST_B8':'motivacion'},inplace=True)
# Se imputa None a NS/NC:
df_grad['motivacion'].replace({9:None},inplace=True)
df_grad['motivacion'].unique() 

array([1, 2, 3, None], dtype=object)

In [137]:
proporciones('motivacion')

3    0.087931
2    0.161403
1    0.750666
Name: motivacion, dtype: float64

In [138]:
# Distribución de la motivación condicionada por la rama de conocimiento de la titulación (1=Artes y humanidades,
# ...  2=Ciencias, 3=Ciencias sociales y jurídicas, 4=Ingeniería y arquitectura, 5=Ciencias de la salud):
dist_condicionada('motivacion','rama')
# La motivación principal son las expectativas laborales para todas las ramas de conocimiento.
# Artes y humanidades es la rama que menos se escoge pensando en las expectativas laborales, seguida por Ciencias.
# También son estas, en ese orden, las que más se escogen para ampliar conocimientos por satisfacción personal. 

motivacion
rama motivacion            
1    1             0.550047
     2             0.363435
     3             0.086517
2    1             0.685259
     2             0.241941
     3             0.072800
3    1             0.791191
     2             0.127498
     3             0.081311
4    1             0.798950
     2             0.111778
     3             0.089272
5    1             0.730134
     2             0.152232
     3             0.117634

#### 'SAT1' 
¿Si tuviera que empezar de nuevo **volvería a cursar estudios universitarios**?  1=Sí, 2=No, 9=NS/NC.  

Nota: Se modifica el valor asociado a la respuesta No: 0=No.

In [139]:
df_grad['SAT1'].unique() # Missing values as '9'

array([1, 2, 9])

In [140]:
# Se imputa None a NS/NC y se modifica respuesta No como 0:
df_grad['SAT1'].replace({9:None,2:0},inplace=True)

In [141]:
proporciones('SAT1',dropna=False)

NaN    0.015671
0.0    0.126410
1.0    0.857919
Name: SAT1, dtype: float64

In [142]:
# Distribución de la disposición a estudiar si tuviera que hacerlo de nuevo según ...
# ... la rama de conocimiento de la titulación:
dist_condicionada('SAT1','rama')
# La disposición a estudiar de nuevo es similar para todas las ramas excepto para los graduados ...
# ... en ciencias de la salud, más dispuestos a estudiar de nuevo que el resto. 

SAT1
rama SAT1          
1    0.0   0.144544
     1.0   0.855456
2    0.0   0.149781
     1.0   0.850219
3    0.0   0.135401
     1.0   0.864599
4    0.0   0.135401
     1.0   0.864599
5    0.0   0.070700
     1.0   0.929300

In [143]:
# Distribución de la disposición a estudiar si tuviera que hacerlo de nuevo según el sexo: 
dist_condicionada('SAT1','fem')
# Las diferencias entre sexos son pequeñas. 

SAT1
fem SAT1          
0   0.0   0.134148
    1.0   0.865852
1   0.0   0.124091
    1.0   0.875909

In [144]:
# Distribución de la disposición a estudiar según las motivaciones para estudiar un grado: 
dist_condicionada('SAT1','motivacion')
# La disposición a estudiar de nuevo más alta se da entre quienes estudiaron ... 
# ... principalmente para ampliar conocimientos por satisfacción personal. 

SAT1
motivacion SAT1          
1          0.0   0.133345
           1.0   0.866655
2          0.0   0.109275
           1.0   0.890725
3          0.0   0.122749
           1.0   0.877251

#### 'SAT2'   
(Solo para respuesta Sí (1) en 'SAT1'?  

¿Si tuviera que empezar de nuevo **volvería a estudiar la misma titulación**?  1=Sí, 2=No, 9=NS/NC.  

Nota: Se modifica el valor asociado a la respuesta No: 0=No.

In [145]:
df_grad['SAT2'].unique() # Missing values as ' ' or '9'

array(['2', '1', ' ', '9'], dtype=object)

In [146]:
df_grad[['SAT1','SAT2']].groupby('SAT1',dropna=False).agg(['unique'])

,SAT2
,unique
SAT1,
0.0,[ ]
1.0,"[2, 1, 9]"
NaN,[ ]


In [147]:
# Se modifica valor asociado a respuesta No (0 en lugar de 2):
df_grad['SAT2'].replace({'2':0},inplace=True)
# Se imputa respuesta No (0) a quienes declararon que no volverían a cursar estudios universitarios: 
df_grad['SAT2'].mask(df_grad['SAT1']==0,0,inplace=True)
# Se imputa None a resto de respuestas en blanco y NS/NC:
df_grad['SAT2'].replace({' ':None,'9':None},inplace=True)
df_grad['SAT2']=df_grad['SAT2'].astype('float')
df_grad[['SAT1','SAT2']].groupby('SAT1',dropna=False).agg(['unique'])

,SAT2
,unique
SAT1,
0.0,[0.0]
1.0,"[0.0, 1.0, nan]"
NaN,[nan]


In [148]:
proporciones('SAT2',dropna=False)
# La disposición a estudiar de nuevo algún grado universitario ('SAT1') es muy superior a la disposición a ...
# ... volver a elegir los mismos estudios. 

NaN    0.019273
0.0    0.365612
1.0    0.615115
Name: SAT2, dtype: float64

In [149]:
# Distribución de la disposición a estudiar la misma titulación si tuviera que hacerlo de nuevo según ...
# ... la rama de conocimiento de la titulación:
dist_condicionada('SAT2','rama')
# La mayor disposición a estudiar de nuevo la misma titulación se da entre los graduados en ciencias de la salud. 

SAT2
rama SAT2          
1    0.0   0.358950
     1.0   0.641050
2    0.0   0.404683
     1.0   0.595317
3    0.0   0.393594
     1.0   0.606406
4    0.0   0.396444
     1.0   0.603556
5    0.0   0.260286
     1.0   0.739714

In [150]:
# Distribución de la disposición a estudiar de nuevo la misma titulación según las motivaciones para ...
# ... estudiar un grado: 
dist_condicionada('SAT2','motivacion')
# Quienes estudiaron principalmente para ampliar conocimientos por satifacción personal se muestran más ...
# ... dispuestos a estudiar de nuevo la misma titulación. 

SAT2
motivacion SAT2          
1          0.0   0.387373
           1.0   0.612627
2          0.0   0.304096
           1.0   0.695904
3          0.0   0.373842
           1.0   0.626158

#### Variables sobre otra formación realizada: variables  'EST_B11_1' a 'EST_B11_7'
Nota: Debe tenerse en cuenta que se pregunta sobre otros estudios completados, no sobre los iniciados, y que para los graduados recientes se pueden estar subestimando las titulaciones posteriores por no haber habido tiempo suficiente para terminarlas. 

#### 'EST_B11_1'  (redesignada como 'varios_grad')
¿Ha completado **otros estudios de grado**, diplomatura, licenciatura o equivalente?  1=Sí, 2=No, 9=NS/NC.  

Nota: Se modifica el valor asociado a la respuesta No: 0=No.

In [151]:
df_grad['EST_B11_1'].unique() # No missing values

array([1, 2])

In [152]:
# Redefinición de la variable: 
df_grad.rename(columns={'EST_B11_1':'varios_grad'},inplace=True)
# Se sustituye valor No por 0:
df_grad['varios_grad'].replace({2:0},inplace=True)

In [153]:
Variables.append('varios_grad')

In [154]:
proporciones('varios_grad')

1    0.234495
0    0.765505
Name: varios_grad, dtype: float64

In [155]:
# Distribución de uno o varios grados según rama de conocimiento de la titulación:
dist_condicionada('varios_grad','rama')
# Los graduados en ciencias sociales y jurídicas, y en ingeniería y arquitectura son los que ...
# ... tienen más de un grado con mayor frecuencia. 
# Los graduados en ciencias de la salud son los que con menos frecuencia tienen más de un grado, a pesar de que ...
# ... también son los que con más frecuencia manifiestan su disposición a estudiar si tuvieran que hacerlo ...
# ... de nuevo (el mismo u otro grado) (variables 'SAT1' y 'SAT2'). 

varios_grad
rama varios_grad             
1    0               0.818697
     1               0.181303
2    0               0.850198
     1               0.149802
3    0               0.719595
     1               0.280405
4    0               0.739448
     1               0.260552
5    0               0.862745
     1               0.137255

In [156]:
# Distribución de la disposición a estudiar si tuvieran que empezar de nuevo según se tengan uno o varios grados: 
dist_condicionada('SAT1','varios_grad')
# La disposición a estudiar si tuvieran que hacerlo de nuevo es algo mayor entre quienes tienen más de un grado. 

SAT1
varios_grad SAT1          
0           0.0   0.137266
            1.0   0.862734
1           0.0   0.099465
            1.0   0.900535

In [157]:
# Distribución de uno o varios grados según el sexo:
dist_condicionada('varios_grad','fem')
# No hay diferencias significativas 

varios_grad
fem varios_grad             
0   0               0.761162
    1               0.238838
1   0               0.768775
    1               0.231225

In [158]:
# Distribución de uno o varios grados según las motivaciones para estudiar: 
dist_condicionada('varios_grad','motivacion')
# Tener más de un grado es más frecuente entre quienes estudiaron principalmente para ampliar conocimientos ...
# ... por satifacción personal que entre quienes estudiaron principalmente por las espectativas profesionales. 

varios_grad
motivacion varios_grad             
1          0               0.785916
           1               0.214084
2          0               0.681729
           1               0.318271
3          0               0.756581
           1               0.243419

#### 'EST_B11_2'  (redesignada como 'master')   
¿Ha completado otros estudios de **máster universitario**?  1=Sí, 2=No, 9=NS/NC.  

Nota: Se modifica el valor asociado a la respuesta No: 0=No.

In [159]:
df_grad['EST_B11_2'].unique() # No missing values

array([2, 1])

In [160]:
# Redefinición de la variable: 
df_grad.rename(columns={'EST_B11_2':'master'},inplace=True)
# Se sustituye valor No por 0:
df_grad['master'].replace({2:0},inplace=True)

In [161]:
Variables.append('master')

In [162]:
df_grad[['varios_grad','master']].mean()
# La proporcion de graduados con un máster universitario ronda el 50 %. 
# La frecuencia de estudios de máster es el doble que la de una segunda titulación de grado. 

varios_grad    0.234495
master         0.474740
dtype: float64

In [163]:
# Distribución de estudios de máster universitario según uno o varios grados:
dist_condicionada('master','varios_grad')
# Es más frecuente tener un máster universitario entre quienes tienen solo un título de grado. 

master
varios_grad master          
0           0       0.499154
            1       0.500846
1           0       0.610482
            1       0.389518

In [164]:
# Distribución de estudios de máster universitario según las motivaciones para estudiar: 
dist_condicionada('master','motivacion')
# Diferencias pequeñas entre quienes declaran motivacion por expectativas profesionales y por ...
# ... adquirir conocimientos.

master
motivacion master          
1          0       0.524902
           1       0.475098
2          0       0.521415
           1       0.478585
3          0       0.546700
           1       0.453300

In [165]:
# Distribución de estudios de máster universitario según rama de conocimiento de la titulación:
dist_condicionada('master','rama')
# Los graduados en ciencias y en artes y humanidades son quienes con mayor frecuencia tienen estudios de máster. 
# La menor frecuencia se da entre los graduados en ingeniería y arquitectura.

master
rama master          
1    0       0.340258
     1       0.659742
2    0       0.291682
     1       0.708318
3    0       0.586252
     1       0.413748
4    0       0.597017
     1       0.402983
5    0       0.496435
     1       0.503565

In [166]:
# Distribución de estudios de máster universitario según el sexo:
dist_condicionada('master','fem')
# Entre los graduados, las mujeres son ligeramente más propensas que los hombres a realizar estudios de máster. 

master
fem master          
0   0       0.526149
    1       0.473851
1   0       0.524590
    1       0.475410

In [167]:
# Distribución del sexo según se tengan o no estudios de máster universitario:
dist_condicionada('master','fem',rev=True)
# Las mujeres también son mayoría entre los graduados. En consecuencia, el número de mujeres con estudios de ...
# ... máster es algo superior al de hombres. 

fem
master fem          
0      0    0.430256
       1    0.569744
1      0    0.428724
       1    0.571276

#### 'EST_B11_3'   (redesignada como 'doctor')  
¿Ha completado otros estudios de **doctorado universitario**?  1=Sí, 2=No, 9=NS/NC.  

Nota: Se modifica el valor asociado a la respuesta No: 0=No.

In [168]:
df_grad['EST_B11_3'].unique() # No missing values

array([2, 1])

In [169]:
# Redefinición de la variable: 
df_grad.rename(columns={'EST_B11_3':'doctor'},inplace=True)
# Se sustituye valor No por 0:
df_grad['doctor'].replace({2:0},inplace=True)

In [170]:
Variables.append('doctor')

In [171]:
df_grad[['varios_grad','master','doctor']].mean()
# Tener un doctorado universitario es mucho menos frecuente que tener un máster universitario o más de un grado. 

varios_grad    0.234495
master         0.474740
doctor         0.015355
dtype: float64

In [172]:
# Distribución de estudios de doctorado según el sexo: 
dist_condicionada('doctor','fem')
# Tener un doctorado es más frecuente entre los hombres graduados que entre las mujeres graduadas. 
# La diferencia es suficiente para contrarestar que hay más mujeres entre los graduados, haciendo que ...
# ... la proporción de doctores sea mayor que la de doctoras. En efecto, (celda siguiente)

doctor
fem doctor          
0   0       0.981022
    1       0.018978
1   0       0.987373
    1       0.012627

In [173]:
dist_condicionada('doctor','fem',rev=True)
# Entre los doctores, la proporción hombres es superior a la de mujeres. 

fem
doctor fem          
0      0    0.427948
       1    0.572052
1      0    0.530864
       1    0.469136

In [174]:
# Distribucion de estudios de doctorado según la motivación para estudiar: 
dist_condicionada('doctor','motivacion')
# La mayor propensión a terminar un doctorado se da entre los que declaran estudiar para adquirir conocimientos

doctor
motivacion doctor          
1          0       0.986820
           1       0.013180
2          0       0.972692
           1       0.027308
3          0       0.990624
           1       0.009376

In [175]:
# Distribucion de las motivaciones para estudiar según se tenga o no el título de doctor: 
dist_condicionada('doctor','motivacion',rev=True)
# La motivación para estudiar más frecuente de quienes terminan un doctorado son las expectativas laborales. 
# Sin embargo, esta motivación es menos frecuente entre los doctores que entre los no doctores. Ocurre lo ...
# ... contrario con la motivación de adquirir conocimientos por satisfacción personal. 

motivacion
doctor motivacion            
0      1             0.752149
       2             0.159406
       3             0.088445
1      1             0.654088
       2             0.291405
       3             0.054507

In [176]:
# Distribucion de estudios de doctorado según las ramas de conocimiento: 
dist_condicionada('doctor','rama')
# La frecuencia más alta de doctores se da entre los graduados en ciencias. También son los que ...
# ... terminan un máster universitario con mayor frecuencia (véase la celda siguiente). 
# Los graduados en ciencias sociales y jurídicas con los menos propensos a terminar un doctorado. También ...
# ... son los segundos menos propensos a estudiar un máster universitario. 
# Para interpretar estos resultados debe tenerse en cuenta la mayor duración de los estudios de doctorado ...
# ... (incluyendo el tiempo dedicado a la tesis). Hace más probable que haya encuestados que cursan estos ...
# ... estudios sin haberlos concluido. 
# Con los datos disponibles no podemos inferir cuántos de los estudios de máster son habilitantes para el ...
# ... doctorado, que podrían considerarse estudios de doctorado no terminados. 

doctor
rama doctor          
1    0       0.979855
     1       0.020145
2    0       0.939143
     1       0.060857
3    0       0.993933
     1       0.006067
4    0       0.985981
     1       0.014019
5    0       0.984180
     1       0.015820

In [177]:
# Distribucion de estudios de máster universitario según las ramas de conocimiento: 
dist_condicionada('master','rama')
# Para facilitar comparación con resultados de la celda anterior. 

master
rama master          
1    0       0.340258
     1       0.659742
2    0       0.291682
     1       0.708318
3    0       0.586252
     1       0.413748
4    0       0.597017
     1       0.402983
5    0       0.496435
     1       0.503565

#### 'EST_B11_4'  (redesignada como 'est_art')
¿Ha completado otros **estudios artísticos superiores** (artes plásticas, diseño, música y danza, arte dramático o conservación y restauración de bienes culturales)?  1=Sí, 2=No, 9=NS/NC.  

Nota: Se modifica el valor asociado a la respuesta No: 0=No.

In [178]:
df_grad['EST_B11_4'].unique() # No missing values

array([2, 1])

In [179]:
# Redefinición de la variable: 
df_grad.rename(columns={'EST_B11_4':'est_art'},inplace=True)
# Se sustituye valor No por 0:
df_grad['est_art'].replace({2:0},inplace=True)

In [180]:
Variables.append('est_art')

In [181]:
df_grad[['varios_grad','master','doctor','est_art']].mean()
# Los estudios artísticos superiores son menos frecuentes que los de doctorado. 

varios_grad    0.234495
master         0.474740
doctor         0.015355
est_art        0.007993
dtype: float64

In [182]:
# Distribución de estudios artísticos superiores según las ramas de conocimiento: 
dist_condicionada('est_art','rama')
# La frecuencia más alta corresponde a los graduados en artes y humanidades (la única superior al 1 %). 

est_art
rama est_art          
1    0        0.973245
     1        0.026755
2    0        0.992798
     1        0.007202
3    0        0.993174
     1        0.006826
4    0        0.995227
     1        0.004773
5    0        0.996212
     1        0.003788

In [183]:
# Distribución de las ramas de conocimiento según se tengan o no estudios artísticos superiores: 
dist_condicionada('est_art','rama',rev=True)
# Sin embargo, la rama de conocimiento más frecuente entre quienes tienen estudios artísticos superiores ...
# ... son las ciencias sociales y jurídicas. Esto se debe a que representan casi la mitad de los ...
# ... graduados.

rama
est_art rama          
0       1     0.098478
        2     0.087808
        3     0.458787
        4     0.212529
        5     0.142398
1       1     0.335968
        2     0.079051
        3     0.391304
        4     0.126482
        5     0.067194

In [184]:
# Distribución de estudios artísticos superiores según las motivaciones para estudiar: 
dist_condicionada('est_art','motivacion')
# La menor frecuencia de estudios artísticos superiores corresponde a quienes estudian para mejorar sus ...
# ... expectativas profesionales. 

est_art
motivacion est_art          
1          0        0.993410
           1        0.006590
2          0        0.987426
           1        0.012574
3          0        0.988460
           1        0.011540

In [185]:
# Distribución de las motivaciones para estudiar según se tengan o no estudios artísticos superiores: 
dist_condicionada('est_art','motivacion',rev=True)
# La motivación mayoritaria para estudiar de quienes tienen estudios artísticos superiores son las ...
# ... expectativas profesionales, aunque en proporción inferior a la general. 

motivacion
est_art motivacion            
0       1             0.751726
        2             0.160657
        3             0.087617
1       1             0.619048
        2             0.253968
        3             0.126984

In [186]:
# Distribución de estudios artísticos superiores según el sexo: 
dist_condicionada('est_art','fem')
# La proporcion de estudios artísticos superiores entre los graduados es mayor para las mujeres 
# ... que para los hombres. En consecuencia, (celda siguiente)

est_art
fem est_art          
0   0        0.993527
    1        0.006473
1   0        0.990862
    1        0.009138

In [187]:
# Distribución del sexo según se tengan o no estudios artísticos superiores: 
dist_condicionada('est_art','fem',rev=True)
# El número de mujeres casi dobla el de hombres entre quienes tienen estudios artísticos superiores. 

fem
est_art fem          
0       0    0.430187
        1    0.569813
1       0    0.347826
        1    0.652174

#### 'EST_B11_5'   (redesignada como 'g_sup')
¿Ha completado algún **ciclo formativo de grado superior** (formación profesional, artes plásticas y diseño) o **enseñanzas deportivas de grado superior**?  1=Sí, 2=No, 9=NS/NC.  

Nota: Se modifica el valor asociado a la respuesta No: 0=No.

In [188]:
df_grad['EST_B11_5'].unique() # No missing values

array([2, 1])

In [189]:
# Redefinición de la variable: 
df_grad.rename(columns={'EST_B11_5':'g_sup'},inplace=True)
# Se sustituye valor No por 0:
df_grad['g_sup'].replace({2:0},inplace=True)

In [190]:
Variables.append('g_sup')

In [191]:
df_grad[['varios_grad','master','doctor','est_art','g_sup']].mean()
# La proporción de graduados con ciclos formativos o enseñanzas deportivas de grado superior es menor ...
# ... que la de un segundo título de grado y muy superior a la de estudios de doctorado. 

varios_grad    0.234495
master         0.474740
doctor         0.015355
est_art        0.007993
g_sup          0.128242
dtype: float64

In [192]:
# Distribución de ciclos formativos y enseñanzas deportivas de grado superior según el sexo: 
dist_condicionada('g_sup','fem')
# La proporcion de ciclos formativos y enseñanzas deportivas de grado superior entre los graduados es mayor ...
# ... para las mujeres que para los hombres. En consecuencia, y puesto que las mujeres son mayoria entre ...
# ... los graduados, (celda siguiente)

g_sup
fem g_sup          
0   0      0.877308
    1      0.122692
1   0      0.867579
    1      0.132421

In [193]:
# Distribución del sexo según se tengan o no ciclos formativos y enseñanzas deportivas de grado superior: 
dist_condicionada('g_sup','fem',rev=True)
# El número de mujeres es superior al de hombres entre quienes tienen ciclos formativos o enseñanzas ...
# ... deportivas de grado superior.

fem
g_sup fem          
0     0    0.432263
      1    0.567737
1     0    0.410939
      1    0.589061

In [194]:
# Distribución de ciclos formativos y enseñanzas deportivas de grado superior según la rama de conocimiento: 
dist_condicionada('g_sup','rama')
# Los graduados en ciencias de la salud y en ciencias sociales y jurídicas son los que con mayor frecuencia ...
# ... han completado ciclos formativos y enseñanzas deportivas de grado superior.

g_sup
rama g_sup          
1    0      0.911237
     1      0.088763
2    0      0.923659
     1      0.076341
3    0      0.855350
     1      0.144650
4    0      0.885757
     1      0.114243
5    0      0.843806
     1      0.156194

In [195]:
# Distribución de ciclos formativos y enseñanzas deportivas de grado superior según doctorado: 
dist_condicionada('g_sup','doctor')
# La presencia de graduados con ciclos formativos y enseñanzas deportivas de grado superior es más de tres ..
# ... veces superior entre los no doctores que entre los doctores. 

g_sup
doctor g_sup          
0      0      0.870367
       1      0.129633
1      0      0.960905
       1      0.039095

In [196]:
# Distribución de ciclos formativos y enseñanzas deportivas de grado superior según estudios de máster: 
dist_condicionada('g_sup','master')
# La presencia de graduados con ciclos formativos y enseñanzas deportivas de grado superior entre quienes ...
# ... no tienen un título de máster. 

g_sup
master g_sup          
0      0      0.830677
       1      0.169323
1      0      0.917210
       1      0.082790

In [197]:
# Distribución de las motivaciones para estudiar según se tengan o no ciclos formativos y enseñanzas ...
# ... deportivas de grado superior: 
dist_condicionada('g_sup','motivacion',rev=True)
# No hay diferencias muy notables. 

motivacion
g_sup motivacion            
0     1             0.753611
      2             0.160646
      3             0.085743
1     1             0.730665
      2             0.166543
      3             0.102792

In [198]:
# Distribución de los ciclos formativos y enseñanzas deportivas de grado superior según la edad: 
dist_condicionada('g_sup','edad')
# El grupo de gradudados de edad de 30 a 34 es el que con mayor frecuencia tiene un título de ciclos formativos o ...
# ... enseñanzas deportivas de grado superior. 
# La diferencia de frecuencia es suficiente para contrarestar que los graduados de menos de 30 años son muchos ...
# ... más. En efecto, (celda siguiente)

g_sup
edad g_sup          
1    0      0.952505
     1      0.047495
2    0      0.782550
     1      0.217450
3    0      0.804186
     1      0.195814

In [199]:
dist_condicionada('g_sup','edad',rev=True)
# La edad más frecuente entre los graduados que tienen un título de ciclos formativos o enseñanzas deportivas ...
# ... de grado superior es el grupo de 30 a 34 años. 

edad
g_sup edad          
0     1     0.542222
      2     0.250290
      3     0.207488
1     1     0.183789
      2     0.472777
      3     0.343434

#### 'EST_B11_6'   (redesignada como 'g_medio')
¿Ha completado algún **ciclo formativo de grado medio** (formación profesional, artes plásticas y diseño) o **enseñanzas deportivas de grado medio**?  1=Sí, 2=No, 9=NS/NC.  

Nota: Se modifica el valor asociado a la respuesta No: 0=No.

In [200]:
df_grad['EST_B11_6'].unique() # No missing values

array([2, 1])

In [201]:
# Redefinición de la variable: 
df_grad.rename(columns={'EST_B11_6':'g_medio'},inplace=True)
# Se sustituye valor No por 0:
df_grad['g_medio'].replace({2:0},inplace=True)

In [202]:
Variables.append('g_medio')

In [203]:
df_grad[['varios_grad','master','doctor','est_art','g_sup','g_medio']].mean()
# Menos frecuentes que los mismos estudios de grado superior, más frecuentes que estudios artisticos superiores ...
# ... y que estudios de doctorado.

varios_grad    0.234495
master         0.474740
doctor         0.015355
est_art        0.007993
g_sup          0.128242
g_medio        0.033332
dtype: float64

In [204]:
# Distribución de los ciclos formativos y educación deportiva de grado medio según esos estudios en su ...
# ... grado superior: 
dist_condicionada('g_medio','g_sup')
# Casi el 90 % de los graduados universitarios que terminan ciclos formativos ...
# ... o educación deportiva de grado superior acceden a esos estudios sin haber cursado los de grado medio. 

g_medio
g_sup g_medio          
0     0        0.977638
      1        0.022362
1     0        0.892092
      1        0.107908

In [205]:
# Distribución de los ciclos formativos y educación deportiva de grado medio según el sexo: 
dist_condicionada('g_medio','fem')
# Las distribuciones condicionadas por el sexo son casi idénticas.  

g_medio
fem g_medio          
0   0        0.966311
    1        0.033689
1   0        0.966936
    1        0.033064

#### 'EST_B11_7'     
No ha completado **ninguno de los estudios anteriores**: 1=Sí, 2=No, 9=NS/NC.  

Nota: Se modifica el valor asociado a la respuesta No: 0=No.

In [206]:
df_grad['EST_B11_7'].unique() # No missing values

array([2, 1])

In [207]:
# Se sustituye valor No por 0:
df_grad['EST_B11_7'].replace({2:0},inplace=True)

In [208]:
df_grad[['varios_grad','master','doctor','est_art','g_sup','g_medio','EST_B11_7']].groupby(
        ['EST_B11_7']).agg(['unique'])
# Comprobación: solo toma valor 1 cuando todas las variables de otros estudios anteriores toman valor 0. 

,varios_grad,master,doctor,est_art,g_sup,g_medio
,unique,unique,unique,unique,unique,unique
EST_B11_7,,,,,,
0,"[1, 0]","[0, 1]","[0, 1]","[0, 1]","[0, 1]","[0, 1]"
1,[0],[0],[0],[0],[0],[0]


#### 'EST_B12' 
(Solo para quienes declaran haber terminado otros estudios de grado, diplomatura, licenciatura o equivalente -respuesta 1 en variable 'varios_grad')  

**Número de otros estudios de grado**, diplomatura, licenciatura o equivalente (aparte de aquel al que se refiere la encuesta): 1=Uno, 2=Dos, 3=Tres o más.  

Nota: Se añade valor 0=No tiene otros títulos de grado.

In [209]:
df_grad['EST_B12'].unique() # Missing values as ' '

array(['2', '1', ' ', '3'], dtype=object)

In [210]:
df_grad[['varios_grad','EST_B12']].groupby('varios_grad',dropna=False).agg(['unique'])

,EST_B12
,unique
varios_grad,
0,[ ]
1,"[2, 1, 3]"


In [211]:
# Se imputa valor 0 a quienes declararon no haber realizado otros estudios de este tipo: 
df_grad['EST_B12'].replace({' ':'0'},inplace=True)
df_grad['EST_B12']=df_grad['EST_B12'].astype('int')
df_grad[['varios_grad','EST_B12']].groupby('varios_grad').agg(['unique'])

,EST_B12
,unique
varios_grad,
0,[0]
1,"[2, 1, 3]"


####  Nueva variable 'n_grad'     
**Número de títulos de grado**, diplomatura, licenciatura o equivalente (incluyendo aquella que motiva esta encuesta): 1=Uno, 2=Dos, 3=Tres, 4=Cuatro o más. 

In [212]:
df_grad['n_grad']=df_grad['EST_B12']+1
df_grad[['n_grad','EST_B12']].groupby('n_grad').agg(['unique']) # Comprobación

,EST_B12
,unique
n_grad,
1,[0]
2,[1]
3,[2]
4,[3]


In [213]:
Variables.append('n_grad')

In [214]:
proporciones('n_grad')

4    0.005182
3    0.025181
2    0.204133
1    0.765505
Name: n_grad, dtype: float64

In [215]:
# Distribución de doctores y no doctores según el número de títulos de grado:  
dist_condicionada('n_grad','doctor',rev=True)
# Terminar los estudios de doctorado es más frecuente cuanto mayor es el número de títulos de grado. 

doctor
n_grad doctor          
1      0       0.986586
       1       0.013414
2      0       0.980808
       1       0.019192
3      0       0.963614
       1       0.036386
4      0       0.951220
       1       0.048780

In [216]:
# Distribución de número de grados universitarios según se tenga o no un título de doctor: 
dist_condicionada('n_grad','doctor')
# Los doctores tienen más de un grado universitario con más frecuencia que los no doctores. En concreto, ...
# ... ser doctor multiplica por más de 2 la probabilidad de tener 3 grados y por 3 la de tener 4 o más títulos ...
# ... de grado (incluyendo aquel al que se refiere la encuesta). 

n_grad
doctor n_grad          
0      1       0.767014
       2       0.203337
       3       0.024643
       4       0.005006
1      1       0.668724
       2       0.255144
       3       0.059671
       4       0.016461

In [217]:
# Distribución de títulos de máster universitario según el número de títulos de grado:  
dist_condicionada('n_grad','master',rev=True)
# Los titulados en un único grado universitario son los que con más frecuencia terminan estudios de máster ...
# ... universitario. Les siguen quienes tienen 4 o más títulos de grado. 

master
n_grad master          
1      0       0.499154
       1       0.500846
2      0       0.608110
       1       0.391890
3      0       0.636136
       1       0.363864
4      0       0.579268
       1       0.420732

In [218]:
# Distribución de número de grados universitarios según se tenga o no un título de máster universitario: 
dist_condicionada('n_grad','master')
# Quienes terminan un máster universitario tienen solo un título de grado con más frecuencia que quienes ...
# ... no tienen un título de máster. 

n_grad
master n_grad          
0      1       0.727459
       2       0.236331
       3       0.030496
       4       0.005714
1      1       0.807600
       2       0.168508
       3       0.019300
       4       0.004592

In [219]:
# Distribución de número de grados universitarios según el sexo: 
dist_condicionada('n_grad','fem')
# Las diferencias entre sexos son pequeñas. 

n_grad
fem n_grad          
0   1       0.761162
    2       0.208533
    3       0.024347
    4       0.005958
1   1       0.768775
    2       0.200820
    3       0.025809
    4       0.004597

In [220]:
# Distribución de número de grados universitarios según la edad: 
dist_condicionada('n_grad','edad')
# Como es de esperar, a medida que aumenta la edad aumenta también la frecuencia de titulados en un número ...
# ... mayor de grados. 

n_grad
edad n_grad          
1    1       0.881072
     2       0.111988
     3       0.005857
     4       0.001082
2    1       0.747082
     2       0.225382
     3       0.024816
     4       0.002720
3    1       0.533361
     2       0.381093
     3       0.068268
     4       0.017278

#### 'EST_B13_AMB1'     
(Solo para quienes declaran haber terminado otros estudios de grado, diplomatura, licenciatura o equivalente -respuesta 1 en variable 'varios_grad')  

**Ambito de estudio del segundo grado**, diplomatura, licenciatura o equivalente en el que está titulado: 0112=Formación de docentes de enseñanza infantil, 0113=Formación de docentes de enseñanza primaria, 011=Educación (Otros estudios), 0211=Técnicas audiovisuales y medios de comunicación, 021=Artes (Otros estudios), 022=Humanidades, 023=Lenguas, 0311=Economía, 0313=Psicología, 031=Ciencias sociales y del comportamiento (Otros estudios), 032=Periodismo y documentación, 0413=Dirección y administración, 041=Negocios y administración (Otros estudios), 042=Derecho, 051=Ciencias de la vida, 052=Medio ambiente, 053=Ciencias químicas, físicas y geológicas, 054=Matemáticas y estadística, 061=Informática, 071=Ingeniería y profesiones afines, 072=Industria manufacturera y producción, 073=Arquitectura y construcción, 081=Agricultura y ganadería, 082=Silvicultura, 084=Veterinaria, 0912=Medicina, 0913=Enfermería, 091=Salud (Otros estudios), 092=Servicios sociales, 1014=Actividades físicas y deportivas, 1015=Viajes, turismo y ocio, 109=Servicios (Otros estudios), 9999=NS/NC.  

Nota: Se añade valor 0=No tiene otros títulos de grado. 

In [221]:
df_grad['EST_B13_AMB1'].sort_values().unique() # Missing values as '    ' or '9999'

array(['    ', '011 ', '0112', '0113', '021 ', '0211', '022 ', '023 ',
       '031 ', '0311', '0313', '032 ', '041 ', '0413', '042 ', '051 ',
       '052 ', '053 ', '054 ', '061 ', '071 ', '072 ', '073 ', '081 ',
       '082 ', '084 ', '091 ', '0912', '0913', '092 ', '1014', '1015',
       '109 ', '9999'], dtype=object)

In [222]:
# Función re_varios_grados: Inspección e imputación de valores de col. condicionadas por respuesta ...
# ... a 'EST_B11_1' (redesignada como 'varios_grad').
def re_varios_grados(col,n,blank=' ',nsnc='9'):
    print(df_grad[[col,'n_grad']].groupby('n_grad').agg(['unique']))
    # Se imputa valor 0 a quienes tienen menos de n títulos de grado: 
    df_grad[col].mask(df_grad['n_grad']<n,0,inplace=True)
    # Se imputa None al resto de respuestas en blanco y NS/NC:
    df_grad[col].replace({blank:None,nsnc:None},inplace=True)
    try:
        df_grad[col]=df_grad[col].astype('int')
    except:
        df_grad[col]=df_grad[col].astype('float')
    print(df_grad[[col,'n_grad']].groupby('n_grad').agg(['unique']))
    return

In [223]:
re_varios_grados('EST_B13_AMB1',2,blank='    ',nsnc='9999')

                                                                                                                                                                                                  EST_B13_AMB1
                                                                                                                                                                                                        unique
n_grad                                                                                                                                                                                                        
1                                                                                                                                                                                                       [    ]
2       [011 , 0113, 0112, 9999, 0313, 092 , 073 , 091 , 023 , 031 , 041 , 042 , 071 , 053 , 1015, 051 , 0413, 032 , 0913, 021 , 022 , 1014, 081 , 054 , 0912, 0211, 0311, 0

In [224]:
# El ámbito de estudio es menos manejable (muchas categorías) que la rama de conocimiento (5 valores). 
# Para identificar la rama que corresponde a cada ámbito: 
df_grad[['rama','AMBITO']].groupby('AMBITO').agg(['value_counts']).head(10)

rama
            value_counts
AMBITO rama             
11     3            1061
       1               1
21     1             887
       4              19
22     1             916
23     1            1149
       3               6
31     3             547
       1             160
32     3             769

#### DataFrame auxiliar df_rama_ambito
Dataframe auxiliar para identificar la rama de conocimiento asociada a cada ámbito de estudios.  

In [225]:
# Cuando hay varias ramas asociadas a un ámbito, elegimos la rama más frecuente (posición 1 en ranking): 
df_ambito_rama=df_grad[['rama','AMBITO']].groupby('AMBITO').agg(['value_counts']
                    ).groupby(['AMBITO']).rank(ascending=False).reset_index()
df_ambito_rama.head(10)

AMBITO rama             
              value_counts
0     11    3          1.0
1     11    1          2.0
2     21    1          1.0
3     21    4          2.0
4     22    1          1.0
5     23    1          1.0
6     23    3          2.0
7     31    3          1.0
8     31    1          2.0
9     32    3          1.0

In [226]:
# Ajuste de índices: de multiíndice a índices simples: 
df_ambito_rama.columns=df_ambito_rama.columns.to_flat_index()
df_ambito_rama.columns

Index([('AMBITO', ''), ('rama', ''), ('rama', 'value_counts')], dtype='object')

In [227]:
# Ajuste de índices: cambio de denominación de columnas: 
df_ambito_rama.rename(columns={df_ambito_rama.columns[0]:'ambito',
                               df_ambito_rama.columns[1]:'rama',
                               df_ambito_rama.columns[2]:'rank'},inplace=True)
df_ambito_rama.columns

Index(['ambito', 'rama', 'rank'], dtype='object')

In [228]:
# Ajuste de índices: columna 'ambito' como índice
df_ambito_rama.set_index('ambito',inplace=True)
df_ambito_rama.head(10)

,rama,rank
ambito,,
11,3,1.0
11,1,2.0
21,1,1.0
21,4,2.0
22,1,1.0
23,1,1.0
23,3,2.0
31,3,1.0
31,1,2.0


In [229]:
# Selección de la rama más frecuente para cada ámbito: 
df_ambito_rama=df_ambito_rama[df_ambito_rama['rank']==1]
del df_ambito_rama['rank']
df_ambito_rama.head(10)

,rama
ambito,
11,3
21,1
22,1
23,1
31,3
32,3
41,3
42,3
51,2


In [230]:
# Para transformar el dataset auxiliar en un diccionario: 
D_ambito_rama=dict()
for ambito in df_ambito_rama.index:
    D_ambito_rama[ambito]=df_ambito_rama['rama'].loc[ambito]

In [231]:
D_ambito_rama.keys()

dict_keys([11, 21, 22, 23, 31, 32, 41, 42, 51, 52, 53, 54, 61, 71, 72, 73, 81, 82, 84, 91, 92, 109, 112, 113, 211, 311, 313, 413, 912, 913, 1014, 1015])

In [232]:
D_ambito_rama.items()

dict_items([(11, 3), (21, 1), (22, 1), (23, 1), (31, 3), (32, 3), (41, 3), (42, 3), (51, 2), (52, 2), (53, 2), (54, 2), (61, 4), (71, 4), (72, 4), (73, 4), (81, 4), (82, 4), (84, 5), (91, 5), (92, 3), (109, 4), (112, 3), (113, 3), (211, 3), (311, 3), (313, 5), (413, 3), (912, 5), (913, 5), (1014, 3), (1015, 3)])

#### Nueva variable 'rama_grad2'
**Rama de conocimiento de la segunda titulación de grado** universitario: 0=Sin segunda titulación de grado, 1=Artes y humanidades, 2=Ciencias, 3=Ciencias sociales y jurídicas, 4=Ingeniería y arquitectura, 5=Ciencias de la salud.

In [233]:
# Columna nueva: rama de conocimiento asociada al ámbito declarado: 
df_grad['rama_grad2']=df_grad['EST_B13_AMB1'].replace(D_ambito_rama)

In [234]:
df_grad[['EST_B13_AMB1','rama_grad2']].groupby('EST_B13_AMB1').agg(['unique']).head() # Comprobación 

,rama_grad2
,unique
EST_B13_AMB1,
0.0,[0.0]
11.0,[3.0]
21.0,[1.0]
22.0,[1.0]
23.0,[1.0]


In [235]:
# Comprobación:
df_grad['rama_grad2'].sort_values().unique()

array([ 0.,  1.,  2.,  3.,  4.,  5., nan])

In [236]:
# Comprobación:
df_grad[['EST_B13_AMB1','rama_grad2']].count()

EST_B13_AMB1    31570
rama_grad2      31570
dtype: int64

In [237]:
Variables.append('rama_grad2')

#### 'EST_B14_1'     
(Solo para quienes declaran haber terminado otros estudios de grado, diplomatura, licenciatura o equivalente -respuesta 1 en variable 'varios_grad')  

**Momento en que terminó la segunda titulación de grado**, diplomatura, licenciatura o equivalente mencionada: 1=Antes de comenzar los estudios de la titulación que finalizó en 2014, 2=Durante la realización de los estudios de la titulación que finalizó en 2014, 3=Después de obtener la titulación que finalizó en 2014, 9=NS/NC.

Nota: Se añade valor 0=No tiene otros títulos de grado. 

In [238]:
df_grad['EST_B14_1'].sort_values().unique() # Missing values as ' ' or '9'

array([' ', '1', '2', '3', '9'], dtype=object)

In [239]:
re_varios_grados('EST_B14_1',2)

           EST_B14_1
              unique
n_grad              
1                [ ]
2       [1, 2, 3, 9]
3       [2, 1, 3, 9]
4          [1, 3, 2]
                   EST_B14_1
                      unique
n_grad                      
1                      [0.0]
2       [1.0, 2.0, 3.0, nan]
3       [2.0, 1.0, 3.0, nan]
4            [1.0, 3.0, 2.0]


#### 'EST_B15_1'     
(Solo para quienes declaran haber terminado otros estudios de grado, diplomatura, licenciatura o equivalente -respuesta 1 en variable 'varios_grad')  

**Dónde estudió la segunda titulación de grado**, diplomatura, licenciatura o equivalente mencionada: 1=Completamente en España, 2=Una parte en España y otra parte en el extranjero, 3=Completamente en el extranjero, 9=NS/NC.

Nota: Se añade valor 0=No tiene otros títulos de grado. 

In [240]:
df_grad['EST_B15_1'].sort_values().unique() # Missing values as ' ' or '9'

array([' ', '1', '2', '3', '9'], dtype=object)

In [241]:
re_varios_grados('EST_B15_1',2)

           EST_B15_1
              unique
n_grad              
1                [ ]
2       [1, 2, 3, 9]
3       [1, 2, 9, 3]
4          [2, 1, 3]
                   EST_B15_1
                      unique
n_grad                      
1                      [0.0]
2       [1.0, 2.0, 3.0, nan]
3       [1.0, 2.0, nan, 3.0]
4            [2.0, 1.0, 3.0]


#### 'EST_B13_AMB2'     
(Solo para quienes declaran haber terminado otros estudios de grado, diplomatura, licenciatura o equivalente -respuesta 1 en variable 'varios_grad')  

**Ambito de estudio del tercer grado**, diplomatura, licenciatura o equivalente en el que está titulado: 0112=Formación de docentes de enseñanza infantil, 0113=Formación de docentes de enseñanza primaria, 011=Educación (Otros estudios), 0211=Técnicas audiovisuales y medios de comunicación, 021=Artes (Otros estudios), 022=Humanidades, 023=Lenguas, 0311=Economía, 0313=Psicología, 031=Ciencias sociales y del comportamiento (Otros estudios), 032=Periodismo y documentación, 0413=Dirección y administración, 041=Negocios y administración (Otros estudios), 042=Derecho, 051=Ciencias de la vida, 052=Medio ambiente, 053=Ciencias químicas, físicas y geológicas, 054=Matemáticas y estadística, 061=Informática, 071=Ingeniería y profesiones afines, 072=Industria manufacturera y producción, 073=Arquitectura y construcción, 081=Agricultura y ganadería, 082=Silvicultura, 084=Veterinaria, 0912=Medicina, 0913=Enfermería, 091=Salud (Otros estudios), 092=Servicios sociales, 1014=Actividades físicas y deportivas, 1015=Viajes, turismo y ocio, 109=Servicios (Otros estudios), 9999=NS/NC.  

Nota: Se añade valor 0=No tiene tres títulos de grado. 

In [242]:
df_grad['EST_B13_AMB2'].sort_values().unique() # Missing values as '    ' or '9999'

array(['    ', '011 ', '0112', '0113', '021 ', '0211', '022 ', '023 ',
       '031 ', '0311', '0313', '032 ', '041 ', '0413', '042 ', '051 ',
       '052 ', '053 ', '054 ', '061 ', '071 ', '072 ', '073 ', '081 ',
       '082 ', '084 ', '091 ', '0912', '0913', '092 ', '1014', '1015',
       '109 ', '9999'], dtype=object)

In [243]:
re_varios_grados('EST_B13_AMB2',3,blank='    ',nsnc='9999')

                                                                                                                                                                                      EST_B13_AMB2
                                                                                                                                                                                            unique
n_grad                                                                                                                                                                                            
1                                                                                                                                                                                           [    ]
2                                                                                                                                                                                           [    ]
3       [032 , 011 , 0112

#### Nueva variable 'rama_grad3'
**Rama de conocimiento de la tercera titulación de grado** universitario: 0=Sin tercera titulación de grado, 1=Artes y humanidades, 2=Ciencias, 3=Ciencias sociales y jurídicas, 4=Ingeniería y arquitectura, 5=Ciencias de la salud.

In [244]:
# Columna nueva: rama de conocimiento asociada al ámbito declarado: 
df_grad['rama_grad3']=df_grad['EST_B13_AMB2'].replace(D_ambito_rama)

In [245]:
Variables.append('rama_grad3')

#### 'EST_B14_2'     
(Solo para quienes declaran haber terminado otros estudios de grado, diplomatura, licenciatura o equivalente -respuesta 1 en variable 'varios_grad')  

**Momento en que terminó la tercera titulación de grado**, diplomatura, licenciatura o equivalente mencionada: 1=Antes de comenzar los estudios de la titulación que finalizó en 2014, 2=Durante la realización de los estudios de la titulación que finalizó en 2014, 3=Después de obtener la titulación que finalizó en 2014, 9=NS/NC.

Nota: Se añade valor 0=No tiene tres títulos de grado. 

In [246]:
df_grad['EST_B14_2'].sort_values().unique() # Missing values as ' ' or '9'

array([' ', '1', '2', '3', '9'], dtype=object)

In [247]:
re_varios_grados('EST_B14_2',3)

           EST_B14_2
              unique
n_grad              
1                [ ]
2                [ ]
3       [2, 1, 3, 9]
4          [1, 3, 2]
                   EST_B14_2
                      unique
n_grad                      
1                      [0.0]
2                      [0.0]
3       [2.0, 1.0, 3.0, nan]
4            [1.0, 3.0, 2.0]


#### 'EST_B15_2'     
(Solo para quienes declaran haber terminado otros estudios de grado, diplomatura, licenciatura o equivalente -respuesta 1 en variable 'varios_grad')  

**Dónde estudió la tercera titulación de grado**, diplomatura, licenciatura o equivalente mencionada: 1=Completamente en España, 2=Una parte en España y otra parte en el extranjero, 3=Completamente en el extranjero, 9=NS/NC.

Nota: Se añade valor 0=No tiene tres títulos de grado. 

In [248]:
df_grad['EST_B15_2'].sort_values().unique() # Missing values as ' ' or '9'

array([' ', '1', '2', '3', '9'], dtype=object)

In [249]:
re_varios_grados('EST_B15_2',3)

           EST_B15_2
              unique
n_grad              
1                [ ]
2                [ ]
3       [1, 2, 9, 3]
4          [1, 2, 3]
                   EST_B15_2
                      unique
n_grad                      
1                      [0.0]
2                      [0.0]
3       [1.0, 2.0, nan, 3.0]
4            [1.0, 2.0, 3.0]


#### 'EST_B13_AMB3'     
(Solo para quienes declaran haber terminado otros estudios de grado, diplomatura, licenciatura o equivalente -respuesta 1 en variable 'varios_grad')  

**Ambito de estudio del cuarto grado**, diplomatura, licenciatura o equivalente en el que está titulado: 0112=Formación de docentes de enseñanza infantil, 0113=Formación de docentes de enseñanza primaria, 011=Educación (Otros estudios), 0211=Técnicas audiovisuales y medios de comunicación, 021=Artes (Otros estudios), 022=Humanidades, 023=Lenguas, 0311=Economía, 0313=Psicología, 031=Ciencias sociales y del comportamiento (Otros estudios), 032=Periodismo y documentación, 0413=Dirección y administración, 041=Negocios y administración (Otros estudios), 042=Derecho, 051=Ciencias de la vida, 052=Medio ambiente, 053=Ciencias químicas, físicas y geológicas, 054=Matemáticas y estadística, 061=Informática, 071=Ingeniería y profesiones afines, 072=Industria manufacturera y producción, 073=Arquitectura y construcción, 081=Agricultura y ganadería, 082=Silvicultura, 084=Veterinaria, 0912=Medicina, 0913=Enfermería, 091=Salud (Otros estudios), 092=Servicios sociales, 1014=Actividades físicas y deportivas, 1015=Viajes, turismo y ocio, 109=Servicios (Otros estudios), 9999=NS/NC.  

Nota: Se añade valor 0=No tiene cuatro títulos de grado. 

In [250]:
df_grad['EST_B13_AMB3'].sort_values().unique() # Missing values as '    ' or '9999'

array(['    ', '011 ', '0112', '0113', '021 ', '022 ', '023 ', '031 ',
       '0311', '032 ', '041 ', '0413', '042 ', '054 ', '061 ', '071 ',
       '072 ', '073 ', '082 ', '091 ', '0913', '1014', '109 ', '9999'],
      dtype=object)

In [251]:
re_varios_grados('EST_B13_AMB3',4,blank='    ',nsnc='9999')

                                                                                                                                      EST_B13_AMB3
                                                                                                                                            unique
n_grad                                                                                                                                            
1                                                                                                                                           [    ]
2                                                                                                                                           [    ]
3                                                                                                                                           [    ]
4       [0113, 011 , 0112, 9999, 021 , 1014, 023 , 022 , 032 , 031 , 0413, 0913, 042 , 072 , 041 , 0311, 071 , 061 , 0

#### Nueva variable 'rama_grad4'
**Rama de conocimiento de la cuarta titulación de grado** universitario: 0=Sin cuarta titulación de grado, 1=Artes y humanidades, 2=Ciencias, 3=Ciencias sociales y jurídicas, 4=Ingeniería y arquitectura, 5=Ciencias de la salud.

In [252]:
# Columna nueva: rama de conocimiento asociada al ámbito declarado: 
df_grad['rama_grad4']=df_grad['EST_B13_AMB3'].replace(D_ambito_rama)

In [253]:
Variables.append('rama_grad4')

#### 'EST_B14_3'     
(Solo para quienes declaran haber terminado otros estudios de grado, diplomatura, licenciatura o equivalente -respuesta 1 en variable 'varios_grad')  

**Momento en que terminó la cuarta titulación de grado**, diplomatura, licenciatura o equivalente mencionada: 1=Antes de comenzar los estudios de la titulación que finalizó en 2014, 2=Durante la realización de los estudios de la titulación que finalizó en 2014, 3=Después de obtener la titulación que finalizó en 2014, 9=NS/NC.

Nota: Se añade valor 0=No tiene cuatro títulos de grado. 

In [254]:
df_grad['EST_B14_3'].sort_values().unique() # Missing values as ' ' or '9'

array([' ', '1', '2', '3'], dtype=object)

In [255]:
re_varios_grados('EST_B14_3',4)

        EST_B14_3
           unique
n_grad           
1             [ ]
2             [ ]
3             [ ]
4       [2, 3, 1]
        EST_B14_3
           unique
n_grad           
1             [0]
2             [0]
3             [0]
4       [2, 3, 1]


#### 'EST_B15_3'     
(Solo para quienes declaran haber terminado otros estudios de grado, diplomatura, licenciatura o equivalente -respuesta 1 en variable 'varios_grad')  

**Dónde estudió la cuarta titulación de grado**, diplomatura, licenciatura o equivalente mencionada: 1=Completamente en España, 2=Una parte en España y otra parte en el extranjero, 3=Completamente en el extranjero, 9=NS/NC.

Nota: Se añade valor 0=No tiene cuatro títulos de grado. 

In [256]:
df_grad['EST_B15_3'].sort_values().unique() # Missing values as ' ' or '9'

array([' ', '1', '2', '3'], dtype=object)

In [257]:
re_varios_grados('EST_B15_3',4)

        EST_B15_3
           unique
n_grad           
1             [ ]
2             [ ]
3             [ ]
4       [1, 3, 2]
        EST_B15_3
           unique
n_grad           
1             [0]
2             [0]
3             [0]
4       [1, 3, 2]


#### 'EST_B16'  (redesignada como 'n_master')   
(Solo para quienes declaran haber terminado estudios de máster -respuesta 1 en variable 'master')  

**Número de títulos de máster** universitario que ha completado: 1=Uno, 2=Dos, 3=Tres o más. 

Nota: Se añade valor 0=No tiene títulos de grado. 

In [258]:
df_grad['EST_B16'].unique() # Missing values as ' '

array([' ', '1', '2', '3'], dtype=object)

In [259]:
df_grad[['EST_B16','master']].groupby('master',dropna=False).agg(['unique'])

,EST_B16
,unique
master,
0,[ ]
1,"[1, 2, 3]"


In [260]:
# Se imputa valor 0 a quienes declaran no tener títulos de máster:
df_grad['EST_B16'].mask(df_grad['master']==0,0,inplace=True)
df_grad['EST_B16']=df_grad['EST_B16'].astype('int')
df_grad[['EST_B16','master']].groupby('master',dropna=False).agg(['unique']) # Comprobación

,EST_B16
,unique
master,
0,[0]
1,"[1, 2, 3]"


In [261]:
# Cambio de denominación: 
df_grad.rename(columns={'EST_B16':'n_master'},inplace=True)

In [262]:
Variables.append('n_master')

In [263]:
Variables

['fem',
 'edad',
 'esp',
 'eu27',
 'art_hum',
 'ciencias',
 'cc_soc_ju',
 'ing_arq',
 'cc_salud',
 'tipo_centro',
 'a_dist',
 'univ_pa',
 'univ_ma',
 'beca',
 'extr',
 'estr_sin_beca',
 'varios_grad',
 'master',
 'doctor',
 'est_art',
 'g_sup',
 'g_medio',
 'n_grad',
 'rama_grad2',
 'rama_grad3',
 'rama_grad4',
 'n_master']